In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb
import seaborn as sns
import datetime as dt
from time import time
pd.set_option('display.max_columns', 999)
from sklearn.metrics import accuracy_score
from mlxtend.evaluate import lift_score
import time

In [2]:
#traindf = pd.read_csv('/Users/Vishy/Files/AVDatahack/CrossSell/RawData/train.csv')
traindf = pd.read_csv('/data/train.csv')
print (traindf.shape)
traindf.head()

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,21,23,26,28,29,30,32,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(300000, 333)


,CUSTOMER_ID,ACT_TYPE,GENDER,LEGAL_ENTITY,OCCUP_ALL_NEW,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,BL_TAG,TWL_TAG,PL_TAG,AL_CNC_TAG,AGRI_TAG,CE_TAG,CV_TAG,EDU_TAG,TL_TAG,OTHER_LOANS_TAG,LAP_TAG,LAS_TAG,EEG_TAG,BBG_TAG,HL_TAG,GL_TAG,AL_CLOSED,PL_CLOSED,BL_CLOSED,TWL_CLOSED,CV_CLOSED,LAP_CLOSED,LAS_CLOSED,EEG_CLOSED,BBG_CLOSED,GL_CLOSED,CC_HOLD_MON_03,CC_HOLD_MON_02,CC_HOLD_MON_01,CC_ACTIVE_MON_03,CC_ACTIVE_MON_02,CC_ACTIVE_MON_01,CC_HOLD,CC_ACTIVE,DC_HOLD,DC_ACTIVE_MON_03,DC_ACTIVE_MON_02,DC_ACTIVE_MON_01,DC_ACTIVE,DC_SPEND_MON_03,DC_SPEND_MON_02,DC_SPEND_MON_01,CC_SPEND_MON_04,CC_SPEND_MON_01,CC_SPEND_MON_02,CC_SPEND_MON_03,DC_SPEND_MON_04,PM_FD_MON_01,PM_FD_MON_02,PM_FD_MON_04,CR_LIMIT,STMT_MON_03,STMT_MON_02,STMT_MON_01,COC_ELIGIBLE,LAP_SCRUB_LIVE,LAS_SCRUB_LIVE,PL_SCRUB_LIVE,GL_SCRUB_LIVE,TWL_SCRUB_LIVE,BL_SCRUB_LIVE,CEL_SCRUB_LIVE,TL_SCRUB_LIVE,AL_SCRUB_LIVE,LAP_SCRUB_CLOSED,LAS_SCRUB_CLOSED,PL_SCRUB_CLOSED,GL_SCRUB_CLOSED,TWL_SCRUB_CLOSED,BL_SCRUB_CLOSED,CEL_SCRUB_CLOSED,TL_SCRUB_CLOSED,AL_SCRUB_CLOSED,MATURITY_GL,MATURITY_LAP,MATURITY_LAS,STMT_CON_DAE_ACTIVE_MON_01,BTE_ACTIVE_MON_01,COC_ACTIVE_MON_01,PL2CC_ACTIVE_MON_01,MER_EMI_ACTIVE_MON_01,TRN_CON_DAE_ACTIVE_MON_01,STMT_CON_DAE_CLOSED_MON_01,BTE_CLOSED_MON_01,COC_CLOSED_MON_01,PL2CC_CLOSED_MON_01,MER_EMI_CLOSED_MON_01,TRN_CON_DAE_CLOSED_MON_01,EOP_BAL_MON_01,AMB_MON_04,AMB_MON_01,AMB_MON_02,AMB_MON_03,NB_MON_01_CNT,NB_MON_02_CNT,NB_MON_03_CNT,CDMAT_MON_01,CDMAT_MON_04,MF_MON_04,MF_MON_01,RD_MON_04,RD_MON_01,FD_MON_04,FD_MON_01,LI_MON_04,LI_MON_01,GI_MON_04,GI_MON_01,CC_TXN_MON_01,CC_SPEND_MON_06,CC_TXN_MON_06,CC_TXN_MON_03,CC_TXN_MON_02,CC_TXN_MON_04,CC_SPEND_MON_05,CC_TXN_MON_05,DC_TXN_MON_04,DC_TXN_MON_01,DC_SPEND_MON_06,DC_TXN_MON_06,DC_SPEND_MON_05,DC_TXN_MON_05,DC_TXN_MON_03,DC_TXN_MON_02,DC_SPEND_MON_04_ENT,DC_SPEND_MON_04_MED,DC_SPEND_MON_04_GRC,DC_SPEND_MON_04_HMD,DC_SPEND_MON_04_HBY,DC_SPEND_MON_04_CARE,DC_SPEND_MON_04_TRL,DC_SPEND_MON_04_RST,DC_SPEND_MON_04_JER,DC_SPEND_MON_04_HTL,DC_SPEND_MON_04_ATM,DC_SPEND_MON_03_ENT,DC_SPEND_MON_03_MED,DC_SPEND_MON_03_HMD,DC_SPEND_MON_03_HBY,DC_SPEND_MON_03_CARE,DC_SPEND_MON_03_TRL,DC_SPEND_MON_03_RST,DC_SPEND_MON_03_JER,DC_SPEND_MON_03_HTL,DC_SPEND_MON_03_ATM,DC_SPEND_MON_02_ENT,DC_SPEND_MON_02_MED,DC_SPEND_MON_02_HMD,DC_SPEND_MON_02_HBY,DC_SPEND_MON_02_CARE,DC_SPEND_MON_02_TRL,DC_SPEND_MON_02_RST,DC_SPEND_MON_02_JER,DC_SPEND_MON_02_HTL,DC_SPEND_MON_02_ATM,DC_SPEND_MON_01_ENT,DC_SPEND_MON_01_MED,DC_SPEND_MON_01_HMD,DC_SPEND_MON_01_HBY,DC_SPEND_MON_01_CARE,DC_SPEND_MON_01_TRL,DC_SPEND_MON_01_RST,DC_SPEND_MON_01_JER,DC_SPEND_MON_01_HTL,DC_SPEND_MON_01_ATM,DC_SPEND_MON_05_ENT,DC_SPEND_MON_05_MED,DC_SPEND_MON_05_HMD,DC_SPEND_MON_05_HBY,DC_SPEND_MON_05_CARE,DC_SPEND_MON_05_TRL,DC_SPEND_MON_05_RST,DC_SPEND_MON_05_JER,DC_SPEND_MON_05_HTL,DC_SPEND_MON_05_ATM,CC_SPEND_MON_04_ENT,CC_SPEND_MON_04_MED,CC_SPEND_MON_04_HMD,CC_SPEND_MON_04_HBY,CC_SPEND_MON_04_CARE,CC_SPEND_MON_04_TRL,CC_SPEND_MON_04_RST,CC_SPEND_MON_04_JER,CC_SPEND_MON_04_HTL,CC_SPEND_MON_04_ATM,CC_SPEND_MON_02_ENT,CC_SPEND_MON_02_MED,CC_SPEND_MON_02_HMD,CC_SPEND_MON_02_HBY,CC_SPEND_MON_02_CARE,CC_SPEND_MON_02_TRL,CC_SPEND_MON_02_RST,CC_SPEND_MON_02_JER,CC_SPEND_MON_02_HTL,CC_SPEND_MON_02_ATM,CC_SPEND_MON_01_ENT,CC_SPEND_MON_01_MED,CC_SPEND_MON_01_HMD,CC_SPEND_MON_01_HBY,CC_SPEND_MON_01_CARE,CC_SPEND_MON_01_TRL,CC_SPEND_MON_01_RST,CC_SPEND_MON_01_JER,CC_SPEND_MON_01_HTL,CC_SPEND_MON_01_ATM,FINANCE_MON_01,JOBS_MON_01,LIFESTYLE_MON_01,INMON_01KET_MON_01,TRAVEL_MON_01,CHANNEL_CLICK_DISP,DESIGNATION_FINAL,TOP14_CITY,TOP9_CITY,PL2CC_ELIGIBLE,D_AMT_L3_MON_06,C_AMT_L3_MON_06,C_COUNT_L3_MON_06,D_COUNT_L3_MON_06,MAX_C_AMT_L3_MON_06,D_AMT_L3_MON_05,C_AMT_L3_MON_05,C_COUNT_L3_MON_05,D_COUNT_L3_MON_05,MAX_C_AMT_L3_MON_05,D_AMT_L3_MON_04,C_AMT_L3_MON_04,C_COUNT_L3_MON_04,D_COUNT_L3_MON_04,MAX_C_AMT_L3_MON_04,D_AMT_L3_MON_03,C_AMT_L3_MON_03,C_COUNT_L3_MON_03,D_COUNT_L3_MON_03,MAX_C_AMT_L3_MON_03,D_AMT_L3_MON_02,C_AMT_L3_MON_02,C_COUNT_L3_MON_02,D_COUNT_L3_MON_02,MAX_C_AMT_L3_

In [3]:
# Only Y columns
traindf['BL_TAG'] = np.where(traindf['BL_TAG']=='Y', 1, 0)
traindf['TWL_TAG'] = np.where(traindf['TWL_TAG']=='Y', 1, 0)
traindf['PL_TAG'] = np.where(traindf['PL_TAG']=='Y', 1, 0)
traindf['AL_CNC_TAG'] = np.where(traindf['AL_CNC_TAG']=='Y', 1, 0)
traindf['AGRI_TAG'] = np.where(traindf['AGRI_TAG']=='Y', 1, 0)
traindf['CE_TAG'] = np.where(traindf['CE_TAG']=='Y', 1, 0)
traindf['CV_TAG'] = np.where(traindf['CV_TAG']=='Y', 1, 0)
traindf['EDU_TAG'] = np.where(traindf['EDU_TAG']=='Y', 1, 0)
traindf['TL_TAG'] = np.where(traindf['TL_TAG']=='Y', 1, 0)
traindf['OTHER_LOANS_TAG'] = np.where(traindf['OTHER_LOANS_TAG']=='Y', 1, 0)
traindf['LAP_TAG'] = np.where(traindf['LAP_TAG']=='Y', 1, 0)
traindf['LAS_TAG'] = np.where(traindf['LAS_TAG']=='Y', 1, 0)
traindf['BBG_TAG'] = np.where(traindf['BBG_TAG']=='Y', 1, 0)
traindf['HL_TAG'] = np.where(traindf['HL_TAG']=='Y', 1, 0)
traindf['GL_TAG'] = np.where(traindf['GL_TAG']=='Y', 1, 0)
traindf['AL_CLOSED'] = np.where(traindf['AL_CLOSED']=='Y', 1, 0)
traindf['PL_CLOSED'] = np.where(traindf['PL_CLOSED']=='Y', 1, 0)
traindf['BL_CLOSED'] = np.where(traindf['BL_CLOSED']=='Y', 1, 0)

In [4]:
# Only y's 2
traindf['TWL_CLOSED'] = np.where(traindf['TWL_CLOSED']=='Y', 1, 0)
traindf['CV_CLOSED'] = np.where(traindf['CV_CLOSED']=='Y', 1, 0)
traindf['LAP_CLOSED'] = np.where(traindf['LAP_CLOSED']=='Y', 1, 0)
traindf['LAS_CLOSED'] = np.where(traindf['LAS_CLOSED']=='Y', 1, 0)
traindf['BBG_CLOSED'] = np.where(traindf['BBG_CLOSED']=='Y', 1, 0)
traindf['GL_CLOSED'] = np.where(traindf['GL_CLOSED']=='Y', 1, 0)
traindf['CC_HOLD_MON_03'] = np.where(traindf['CC_HOLD_MON_03']=='Y', 1, 0)
traindf['CC_HOLD_MON_02'] = np.where(traindf['CC_HOLD_MON_02']=='Y', 1, 0)
traindf['CC_HOLD_MON_01'] = np.where(traindf['CC_HOLD_MON_01']=='Y', 1, 0)
traindf['CC_ACTIVE_MON_03'] = np.where(traindf['CC_ACTIVE_MON_03']=='Y', 1, 0)
traindf['CC_ACTIVE_MON_02'] = np.where(traindf['CC_ACTIVE_MON_02']=='Y', 1, 0)
traindf['CC_ACTIVE_MON_01'] = np.where(traindf['CC_ACTIVE_MON_01']=='Y', 1, 0)
traindf['DC_ACTIVE_MON_03'] = np.where(traindf['DC_ACTIVE_MON_03']=='Y', 1, 0)
traindf['DC_ACTIVE_MON_02'] = np.where(traindf['DC_ACTIVE_MON_02']=='Y', 1, 0)
traindf['DC_ACTIVE_MON_01'] = np.where(traindf['DC_ACTIVE_MON_01']=='Y', 1, 0)
traindf['BTE_ACTIVE_MON_01'] = np.where(traindf['BTE_ACTIVE_MON_01']=='Y', 1, 0)
traindf['COC_ACTIVE_MON_01'] = np.where(traindf['COC_ACTIVE_MON_01']=='Y', 1, 0)
traindf['PL2CC_ACTIVE_MON_01'] = np.where(traindf['PL2CC_ACTIVE_MON_01']=='Y', 1, 0)
traindf['MER_EMI_ACTIVE_MON_01'] = np.where(traindf['MER_EMI_ACTIVE_MON_01']=='Y', 1, 0)

In [5]:
# Only y's 3
traindf['TRN_CON_DAE_ACTIVE_MON_01'] = np.where(traindf['TRN_CON_DAE_ACTIVE_MON_01']=='Y', 1, 0)
traindf['BTE_CLOSED_MON_01'] = np.where(traindf['BTE_CLOSED_MON_01']=='Y', 1, 0)
traindf['COC_CLOSED_MON_01'] = np.where(traindf['COC_CLOSED_MON_01']=='Y', 1, 0)
traindf['PL2CC_CLOSED_MON_01'] = np.where(traindf['PL2CC_CLOSED_MON_01']=='Y', 1, 0)
traindf['TRN_CON_DAE_CLOSED_MON_01'] = np.where(traindf['TRN_CON_DAE_CLOSED_MON_01']=='Y', 1, 0)
traindf['LAST_3MTHS_INQ'] = np.where(traindf['LAST_3MTHS_INQ']=='Y', 1, 0)
traindf['PREV_3MTHS_INQ'] = np.where(traindf['PREV_3MTHS_INQ']=='Y', 1, 0)
traindf['CREDIT_ACT'] = np.where(traindf['CREDIT_ACT']=='Y', 1, 0)
traindf['DEBIT_ACT'] = np.where(traindf['DEBIT_ACT']=='Y', 1, 0)
traindf['CEL_TAG'] = np.where(traindf['CEL_TAG']=='Y', 1, 0)
traindf['EL_TAG'] = np.where(traindf['EL_TAG']=='Y', 1, 0)
traindf['AGL_TAG'] = np.where(traindf['AGL_TAG']=='Y', 1, 0)
traindf['CVL_TAG'] = np.where(traindf['CVL_TAG']=='Y', 1, 0)
traindf['AL_TAG'] = np.where(traindf['AL_TAG']=='Y', 1, 0)

In [6]:
# convert non 1 to 0
traindf['LAP_SCRUB_LIVE'] = np.where(traindf['LAP_SCRUB_LIVE']==1.0, 1, 0)
traindf['LAS_SCRUB_LIVE'] = np.where(traindf['LAS_SCRUB_LIVE']==1.0, 1, 0)
traindf['GL_SCRUB_LIVE'] = np.where(traindf['GL_SCRUB_LIVE']==1.0, 1, 0)
traindf['TWL_SCRUB_LIVE'] = np.where(traindf['TWL_SCRUB_LIVE']==1.0, 1, 0)
traindf['CEL_SCRUB_LIVE'] = np.where(traindf['CEL_SCRUB_LIVE']==1.0, 1, 0)
traindf['TL_SCRUB_LIVE'] = np.where(traindf['TL_SCRUB_LIVE']==1.0, 1, 0)
traindf['AL_SCRUB_LIVE'] = np.where(traindf['AL_SCRUB_LIVE']==1.0, 1, 0)
traindf['PL_SCRUB_LIVE'] = np.where(traindf['PL_SCRUB_LIVE']==1.0, 1, 0)
traindf['BL_SCRUB_LIVE'] = np.where(traindf['BL_SCRUB_LIVE']==1.0, 1, 0)
traindf['LAP_SCRUB_CLOSED'] = np.where(traindf['LAP_SCRUB_CLOSED']==1.0, 1, 0)
traindf['LAS_SCRUB_CLOSED'] = np.where(traindf['LAS_SCRUB_CLOSED']==1.0, 1, 0)
traindf['GL_SCRUB_CLOSED'] = np.where(traindf['GL_SCRUB_CLOSED']==1.0, 1, 0)
traindf['TWL_SCRUB_CLOSED'] = np.where(traindf['TWL_SCRUB_CLOSED']==1.0, 1, 0)
traindf['CEL_SCRUB_CLOSED'] = np.where(traindf['CEL_SCRUB_CLOSED']==1.0, 1, 0)
traindf['TL_SCRUB_CLOSED'] = np.where(traindf['TL_SCRUB_CLOSED']==1.0, 1, 0)
traindf['AL_SCRUB_CLOSED'] = np.where(traindf['AL_SCRUB_CLOSED']==1.0, 1, 0)
traindf['PL_SCRUB_CLOSED'] = np.where(traindf['PL_SCRUB_CLOSED']==1.0, 1, 0)
traindf['BL_SCRUB_CLOSED'] = np.where(traindf['BL_SCRUB_CLOSED']==1.0, 1, 0)

In [7]:
# Non useful features
del traindf['OCCUP_ALL_NEW'],traindf['EEG_TAG'],traindf['EEG_CLOSED']

In [8]:
traindf.shape

(300000, 330)

In [9]:
# Identify if it is BUSINESS OR AGRI BUSINESS MAN
traindf['NONBUSNONAGRI'] = np.where((traindf['BL_TAG']+traindf['AGRI_TAG']+traindf['CE_TAG']+traindf['CV_TAG']+
                                     traindf['TL_TAG']+traindf['BBG_TAG']+traindf['BL_SCRUB_LIVE']+
                                     traindf['CEL_SCRUB_LIVE']+traindf['TL_SCRUB_LIVE'])>= 1,1,0)

In [10]:
# New variables related to Loan to Eliminate many varaibles by taking the count
# All variables starting with GB is BANK OF GARDENIA are within the bank
traindf['NONBUSNONAGRI'] = np.where((traindf['BL_TAG']+traindf['AGRI_TAG']+traindf['CE_TAG']+traindf['CV_TAG']+
                                     traindf['TL_TAG']+traindf['BBG_TAG']+traindf['BL_SCRUB_LIVE']+
                                     traindf['CEL_SCRUB_LIVE']+traindf['TL_SCRUB_LIVE'])>= 1,1,0)
traindf['GBSCLN'] = (traindf['TWL_TAG']+traindf['AL_CNC_TAG']+traindf['CE_TAG']+traindf['CV_TAG']+
                        traindf['TL_TAG']+traindf['LAP_TAG']+traindf['LAS_TAG']+traindf['BBG_TAG']+
                        traindf['HL_TAG']+traindf['GL_TAG'])
traindf['GBUNSCLN'] = (traindf['BL_TAG']+traindf['PL_TAG']+traindf['AGRI_TAG']+traindf['EDU_TAG']+
                          traindf['OTHER_LOANS_TAG'])
traindf['GBPLLN'] = (traindf['PL_TAG'])
traindf['GBSCCLSDLN'] = (traindf['AL_CLOSED']+traindf['TWL_CLOSED']+traindf['CV_CLOSED']+traindf['LAP_CLOSED']+
                         traindf['LAS_CLOSED']+traindf['BBG_CLOSED']+traindf['GL_CLOSED'])
traindf['GBUNSCCLSDLN'] = (traindf['PL_CLOSED']+traindf['BL_CLOSED'])
traindf['GBPLLNCLSD'] = (traindf['PL_CLOSED'])
# All variables starting with OB are other Bank details

traindf['OBSCLN'] = (traindf['LAP_SCRUB_LIVE']+traindf['LAS_SCRUB_LIVE']+traindf['GL_SCRUB_LIVE']+
                     traindf['TWL_SCRUB_LIVE']+traindf['CEL_SCRUB_LIVE']+traindf['TL_SCRUB_LIVE']+
                     traindf['AL_SCRUB_LIVE'])
traindf['OBUNSCLN'] = (traindf['PL_SCRUB_LIVE']+traindf['BL_SCRUB_LIVE'])
traindf['OBBPLLN'] = (traindf['PL_SCRUB_LIVE'])
traindf['OBSCLNCLSD'] = (traindf['LAP_SCRUB_CLOSED']+traindf['LAS_SCRUB_CLOSED']+traindf['GL_SCRUB_CLOSED']+
                        traindf['TWL_SCRUB_CLOSED']+traindf['CEL_SCRUB_CLOSED']+traindf['TL_SCRUB_CLOSED']+
                        traindf['AL_SCRUB_CLOSED'])
traindf['OBUNSCLNCLSD'] = (traindf['PL_SCRUB_CLOSED']+traindf['BL_SCRUB_CLOSED'])
traindf['OBPLLNCLSD'] = (traindf['PL_SCRUB_CLOSED'])

In [11]:
# Delete the Unwanted variables now
del traindf['BL_TAG'],traindf['TWL_TAG'],traindf['PL_TAG'],traindf['AL_CNC_TAG'],traindf['AGRI_TAG']
del traindf['CE_TAG'],traindf['CV_TAG'],traindf['EDU_TAG'],traindf['TL_TAG'],traindf['OTHER_LOANS_TAG']
del traindf['LAP_TAG'],traindf['LAS_TAG'],traindf['BBG_TAG'],traindf['HL_TAG'],traindf['GL_TAG']
del traindf['AL_CLOSED'],traindf['PL_CLOSED'],traindf['BL_CLOSED'],traindf['TWL_CLOSED']
del traindf['CV_CLOSED'],traindf['LAP_CLOSED'],traindf['LAS_CLOSED'],traindf['BBG_CLOSED'],traindf['GL_CLOSED']
del traindf['LAP_SCRUB_LIVE'],traindf['LAS_SCRUB_LIVE'],traindf['GL_SCRUB_LIVE'],traindf['TWL_SCRUB_LIVE']
del traindf['CEL_SCRUB_LIVE'],traindf['TL_SCRUB_LIVE'],traindf['AL_SCRUB_LIVE'],traindf['PL_SCRUB_LIVE']
del traindf['BL_SCRUB_LIVE'],traindf['LAP_SCRUB_CLOSED'],traindf['LAS_SCRUB_CLOSED'],traindf['GL_SCRUB_CLOSED']
del traindf['TWL_SCRUB_CLOSED'],traindf['CEL_SCRUB_CLOSED'],traindf['TL_SCRUB_CLOSED'],traindf['AL_SCRUB_CLOSED']
del traindf['PL_SCRUB_CLOSED'],traindf['BL_SCRUB_CLOSED']

In [12]:
traindf.shape

(300000, 301)

In [13]:
# MATURITY_GL, MATURITY_LAP, MATURITY_LAS, CLOSED_DATE
traindf['TODAY'] = time.strftime("%d%b%Y")
traindf['MGLDAYS'] = pd.to_datetime(traindf['MATURITY_GL'],format='%d%b%Y' )-pd.to_datetime(traindf['TODAY'], format='%d%b%Y')
traindf['MLAPDAYS'] = pd.to_datetime(traindf['MATURITY_LAP'],format='%d%b%Y' )-pd.to_datetime(traindf['TODAY'], format='%d%b%Y')
traindf['MLASDAYS'] = pd.to_datetime(traindf['MATURITY_LAS'],format='%d%b%Y' )-pd.to_datetime(traindf['TODAY'], format='%d%b%Y')
traindf['LLOANDAYS'] = pd.to_datetime(traindf['CLOSED_DATE'],format='%d%b%Y' )-pd.to_datetime(traindf['TODAY'], format='%d%b%Y')

In [14]:
# Convert the days into Numbers
traindf['MGLDAYS'] = traindf['MGLDAYS'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)
traindf['MLAPDAYS'] = traindf['MLAPDAYS'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)
traindf['MLASDAYS'] = traindf['MLASDAYS'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)
traindf['LLOANDAYS'] = traindf['LLOANDAYS'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)

In [15]:
# Everything less than 0 should be made 0
traindf['MGLDAYS'] = np.where(traindf['MGLDAYS'] <= 0,0,traindf.MGLDAYS)
traindf['MLAPDAYS'] = np.where(traindf['MLAPDAYS'] <= 0,0,traindf.MLAPDAYS)
traindf['MLASDAYS'] = np.where(traindf['MLASDAYS'] <= 0,0,traindf.MLASDAYS)
traindf['LLOANDAYS'] = np.where(traindf['LLOANDAYS'] <= 0,0,traindf.LLOANDAYS)

In [16]:
# replace all NaN with 0
traindf.MGLDAYS.fillna(0, inplace=True)
traindf.MLAPDAYS.fillna(0, inplace=True)
traindf.MLASDAYS.fillna(0, inplace=True)
traindf.LLOANDAYS.fillna(0, inplace=True)

In [17]:
del traindf['MATURITY_GL'],traindf['MATURITY_LAP'],traindf['MATURITY_LAS'],traindf['CLOSED_DATE'],traindf['TODAY']

In [18]:
traindf.shape

(300000, 301)

In [19]:
#Group Invest snd insurance
traindf['TOTINVEST04'] = traindf.fillna(0)['CDMAT_MON_04']+traindf.fillna(0)['MF_MON_04']+traindf.fillna(0)['RD_MON_04']+traindf.fillna(0)['FD_MON_04']
traindf['TOTINVEST01'] = traindf.fillna(0)['CDMAT_MON_01']+traindf.fillna(0)['MF_MON_01']+traindf.fillna(0)['RD_MON_01']+traindf.fillna(0)['FD_MON_01']
traindf['TOTINS04'] = traindf.fillna(0)['LI_MON_04']+traindf.fillna(0)['GI_MON_04']
traindf['TOTINS01'] = traindf.fillna(0)['LI_MON_01']+traindf.fillna(0)['GI_MON_01']

del traindf['MF_MON_04'],traindf['RD_MON_04'],traindf['FD_MON_04'],traindf['MF_MON_01'],traindf['RD_MON_01'],traindf['CDMAT_MON_04']
del traindf['FD_MON_01'],traindf['LI_MON_04'],traindf['GI_MON_04'],traindf['LI_MON_01'],traindf['GI_MON_01'],traindf['CDMAT_MON_01']
traindf.shape

(300000, 293)

In [23]:
# Credit Card mapping

#traindf['CCYESACT03'] = traindf.apply(lambda x: 1 if x['CC_HOLD_MON_03'] == x['CC_ACTIVE_MON_03'] else 0, axis=1)
#traindf['CCYESACT02'] = traindf.apply(lambda x: 1 if x['CC_HOLD_MON_02'] == x['CC_ACTIVE_MON_02'] else 0, axis=1)
#traindf['CCYESACT01'] = traindf.apply(lambda x: 1 if x['CC_HOLD_MON_01'] == x['CC_ACTIVE_MON_01'] else 0, axis=1)

del traindf['CC_HOLD_MON_03'],traindf['CC_ACTIVE_MON_03'],traindf['CC_HOLD_MON_02'],traindf['CC_ACTIVE_MON_02']
del traindf['CC_HOLD_MON_01'],traindf['CC_ACTIVE_MON_01']
traindf.shape

(300000, 290)

In [24]:
# Credit vs Debit spend
traindf['DBVSCR06'] = np.where((traindf.fillna(0)['CC_SPEND_MON_06']-traindf.fillna(0)['DC_SPEND_MON_06'])>=0,1,0)
traindf['DBVSCR05'] = np.where((traindf.fillna(0)['CC_SPEND_MON_05']-traindf.fillna(0)['DC_SPEND_MON_05'])>=0,1,0)
traindf['DBVSCR04'] = np.where((traindf.fillna(0)['CC_SPEND_MON_04']-traindf.fillna(0)['DC_SPEND_MON_04'])>=0,1,0)
traindf['DBVSCR03'] = np.where((traindf.fillna(0)['CC_SPEND_MON_03']-traindf.fillna(0)['DC_SPEND_MON_03'])>=0,1,0)
traindf['DBVSCR02'] = np.where((traindf.fillna(0)['CC_SPEND_MON_02']-traindf.fillna(0)['DC_SPEND_MON_02'])>=0,1,0)
traindf['DBVSCR01'] = np.where((traindf.fillna(0)['CC_SPEND_MON_01']-traindf.fillna(0)['DC_SPEND_MON_01'])>=0,1,0)

# Count of credit higher than debit
traindf['CONTCCDC'] = traindf['DBVSCR06']+traindf['DBVSCR05']+traindf['DBVSCR04']+traindf['DBVSCR03']+traindf['DBVSCR02']+traindf['DBVSCR01']
del traindf['DBVSCR06'],traindf['DBVSCR05'],traindf['DBVSCR04'],traindf['DBVSCR03'],traindf['DBVSCR02'],traindf['DBVSCR01']

# Credit vs Salary Spend
traindf['SAVSCR03'] = np.where((traindf.fillna(0)['CC_SPEND_MON_03']-traindf.fillna(0)['SAL_MON_03'])>=0,1,0)
traindf['SAVSCR02'] = np.where((traindf.fillna(0)['CC_SPEND_MON_02']-traindf.fillna(0)['SAL_MON_02'])>=0,1,0)
traindf['SAVSCR01'] = np.where((traindf.fillna(0)['CC_SPEND_MON_01']-traindf.fillna(0)['SAL_MON_01'])>=0,1,0)

# Count of Credit more than Salary
traindf['CONTCCSL'] = traindf['SAVSCR03']+traindf['SAVSCR02']+traindf['SAVSCR01']
del traindf['SAVSCR03'],traindf['SAVSCR02'],traindf['SAVSCR01']

# DEBIT AMOUNT VS CREDIT AMOUNT
traindf['DBMINCR06'] = np.where((traindf.fillna(0)['D_AMT_L3_MON_06']-traindf.fillna(0)['C_AMT_L3_MON_06'])>=0,1,0)
traindf['DBMINCR05'] = np.where((traindf.fillna(0)['D_AMT_L3_MON_05']-traindf.fillna(0)['C_AMT_L3_MON_05'])>=0,1,0)
traindf['DBMINCR04'] = np.where((traindf.fillna(0)['D_AMT_L3_MON_04']-traindf.fillna(0)['C_AMT_L3_MON_04'])>=0,1,0)
traindf['DBMINCR03'] = np.where((traindf.fillna(0)['D_AMT_L3_MON_03']-traindf.fillna(0)['C_AMT_L3_MON_03'])>=0,1,0)
traindf['DBMINCR02'] = np.where((traindf.fillna(0)['D_AMT_L3_MON_02']-traindf.fillna(0)['C_AMT_L3_MON_02'])>=0,1,0)
traindf['DBMINCR01'] = np.where((traindf.fillna(0)['D_AMT_L3_MON_01']-traindf.fillna(0)['C_AMT_L3_MON_01'])>=0,1,0)

# Count of Credit higher than Debit
traindf['DEBMORCRE'] = traindf['DBMINCR06']+traindf['DBMINCR05']+traindf['DBMINCR04']+traindf['DBMINCR03']+traindf['DBMINCR02']+traindf['DBMINCR01']
del traindf['DBMINCR06'],traindf['DBMINCR05'],traindf['DBMINCR04'],traindf['DBMINCR03'],traindf['DBMINCR02'],traindf['DBMINCR01']

# Check shape
traindf.shape

(300000, 293)

In [25]:
# % Spend Against Credit Limit
traindf['CRLIM03'] = traindf.fillna(0)['STMT_MON_03']/traindf.fillna(0)['CR_LIMIT']
traindf['CRLIM02'] = traindf.fillna(0)['STMT_MON_02']/traindf.fillna(0)['CR_LIMIT']
traindf['CRLIM01'] = traindf.fillna(0)['STMT_MON_01']/traindf.fillna(0)['CR_LIMIT']
traindf['CRLIM03'] = traindf.CRLIM03.fillna(0)
traindf['CRLIM02'] = traindf.CRLIM02.fillna(0)
traindf['CRLIM01'] = traindf.CRLIM01.fillna(0)
traindf.shape

(300000, 296)

In [26]:
# CC & DC Spend Categorize
#CC SPEND

traindf['CCGODSPN04'] = traindf.fillna(0)['CC_SPEND_MON_04_MED']+traindf.fillna(0)['CC_SPEND_MON_04_HMD']+traindf.fillna(0)['CC_SPEND_MON_04_ATM']
traindf['CCBADSPN04'] = (traindf.fillna(0)['CC_SPEND_MON_04_ENT']+traindf.fillna(0)['CC_SPEND_MON_04_RST']+traindf.fillna(0)['CC_SPEND_MON_04_HTL']+
                         traindf.fillna(0)['CC_SPEND_MON_04_HBY']+traindf.fillna(0)['CC_SPEND_MON_04_CARE']+traindf.fillna(0)['CC_SPEND_MON_04_TRL']+
                         traindf.fillna(0)['CC_SPEND_MON_04_JER'])

traindf['CCGODSPN03'] = traindf.fillna(0)['CC_SPEND_MON_03_MED']+traindf.fillna(0)['CC_SPEND_MON_03_HMD']+traindf.fillna(0)['CC_SPEND_MON_03_ATM']
traindf['CCBADSPN03'] = (traindf.fillna(0)['CC_SPEND_MON_03_ENT']+traindf.fillna(0)['CC_SPEND_MON_03_RST']+traindf.fillna(0)['CC_SPEND_MON_03_HTL']+
                         traindf.fillna(0)['CC_SPEND_MON_03_HBY']+traindf.fillna(0)['CC_SPEND_MON_03_CARE']+traindf.fillna(0)['CC_SPEND_MON_03_TRL']+
                         traindf.fillna(0)['CC_SPEND_MON_03_JER'])

traindf['CCGODSPN02'] = traindf.fillna(0)['CC_SPEND_MON_02_MED']+traindf.fillna(0)['CC_SPEND_MON_02_HMD']+traindf.fillna(0)['CC_SPEND_MON_02_ATM']
traindf['CCBADSPN02'] = (traindf.fillna(0)['CC_SPEND_MON_02_ENT']+traindf.fillna(0)['CC_SPEND_MON_02_RST']+traindf.fillna(0)['CC_SPEND_MON_02_HTL']+
                         traindf.fillna(0)['CC_SPEND_MON_02_HBY']+traindf.fillna(0)['CC_SPEND_MON_02_CARE']+traindf.fillna(0)['CC_SPEND_MON_02_TRL']+
                         traindf.fillna(0)['CC_SPEND_MON_02_JER'])

traindf['CCGODSPN01'] = traindf.fillna(0)['CC_SPEND_MON_01_MED']+traindf.fillna(0)['CC_SPEND_MON_01_HMD']+traindf.fillna(0)['CC_SPEND_MON_01_ATM']
traindf['CCBADSPN01'] = (traindf.fillna(0)['CC_SPEND_MON_01_ENT']+traindf.fillna(0)['CC_SPEND_MON_01_RST']+traindf.fillna(0)['CC_SPEND_MON_01_HTL']+
                         traindf.fillna(0)['CC_SPEND_MON_01_HBY']+traindf.fillna(0)['CC_SPEND_MON_01_CARE']+traindf.fillna(0)['CC_SPEND_MON_01_TRL']+
                         traindf.fillna(0)['CC_SPEND_MON_01_JER'])
# DC SPEND

traindf['DCGODSPN04'] = traindf.fillna(0)['DC_SPEND_MON_04_MED']+traindf.fillna(0)['DC_SPEND_MON_04_HMD']+traindf.fillna(0)['DC_SPEND_MON_04_ATM']
traindf['DCBADSPN04'] = (traindf.fillna(0)['DC_SPEND_MON_04_ENT']+traindf.fillna(0)['DC_SPEND_MON_04_RST']+traindf.fillna(0)['DC_SPEND_MON_04_HTL']+
                         traindf.fillna(0)['DC_SPEND_MON_04_HBY']+traindf.fillna(0)['DC_SPEND_MON_04_CARE']+traindf.fillna(0)['DC_SPEND_MON_04_TRL']+
                         traindf.fillna(0)['DC_SPEND_MON_04_JER'])

traindf['DCGODSPN03'] = traindf.fillna(0)['DC_SPEND_MON_03_MED']+traindf.fillna(0)['DC_SPEND_MON_03_HMD']+traindf.fillna(0)['DC_SPEND_MON_03_ATM']
traindf['DCBADSPN03'] = (traindf.fillna(0)['DC_SPEND_MON_03_ENT']+traindf.fillna(0)['DC_SPEND_MON_03_RST']+traindf.fillna(0)['DC_SPEND_MON_03_HTL']+
                         traindf.fillna(0)['DC_SPEND_MON_03_HBY']+traindf.fillna(0)['DC_SPEND_MON_03_CARE']+traindf.fillna(0)['DC_SPEND_MON_03_TRL']+
                         traindf.fillna(0)['DC_SPEND_MON_03_JER'])

traindf['DCGODSPN02'] = traindf.fillna(0)['DC_SPEND_MON_02_MED']+traindf.fillna(0)['DC_SPEND_MON_02_HMD']+traindf.fillna(0)['DC_SPEND_MON_02_ATM']
traindf['DCBADSPN02'] = (traindf.fillna(0)['DC_SPEND_MON_02_ENT']+traindf.fillna(0)['DC_SPEND_MON_02_RST']+traindf.fillna(0)['DC_SPEND_MON_02_HTL']+
                         traindf.fillna(0)['DC_SPEND_MON_02_HBY']+traindf.fillna(0)['DC_SPEND_MON_02_CARE']+traindf.fillna(0)['DC_SPEND_MON_02_TRL']+
                         traindf.fillna(0)['DC_SPEND_MON_02_JER'])

traindf['DCGODSPN01'] = traindf.fillna(0)['DC_SPEND_MON_01_MED']+traindf.fillna(0)['DC_SPEND_MON_01_HMD']+traindf.fillna(0)['DC_SPEND_MON_01_ATM']
traindf['DCBADSPN01'] = (traindf.fillna(0)['DC_SPEND_MON_01_ENT']+traindf.fillna(0)['DC_SPEND_MON_01_RST']+traindf.fillna(0)['DC_SPEND_MON_01_HTL']+
                         traindf.fillna(0)['DC_SPEND_MON_01_HBY']+traindf.fillna(0)['DC_SPEND_MON_01_CARE']+traindf.fillna(0)['DC_SPEND_MON_01_TRL']+
                         traindf.fillna(0)['DC_SPEND_MON_01_JER'])

In [27]:
#Final 04
traindf['CCGODPER04'] = ((traindf['CCGODSPN04'])/(traindf['CCGODSPN04']+traindf['CCBADSPN04']+traindf['DCGODSPN04']+traindf['DCBADSPN04'])).replace(np.inf, 0)
traindf['CCBADPER04'] = ((traindf['CCBADSPN04'])/(traindf['CCGODSPN04']+traindf['CCBADSPN04']+traindf['DCGODSPN04']+traindf['DCBADSPN04'])).replace(np.inf, 0)
traindf['DCGODPER04'] = ((traindf['DCGODSPN04'])/(traindf['CCGODSPN04']+traindf['CCBADSPN04']+traindf['DCGODSPN04']+traindf['DCBADSPN04'])).replace(np.inf, 0)
traindf['DCBADPER04'] = ((traindf['DCBADSPN04'])/(traindf['CCGODSPN04']+traindf['CCBADSPN04']+traindf['DCGODSPN04']+traindf['DCBADSPN04'])).replace(np.inf, 0)

#Final 03
traindf['CCGODPER03'] = ((traindf['CCGODSPN03'])/(traindf['CCGODSPN03']+traindf['CCBADSPN03']+traindf['DCGODSPN03']+traindf['DCBADSPN03'])).replace(np.inf, 0)
traindf['CCBADPER03'] = ((traindf['CCBADSPN03'])/(traindf['CCGODSPN03']+traindf['CCBADSPN03']+traindf['DCGODSPN03']+traindf['DCBADSPN03'])).replace(np.inf, 0)
traindf['DCGODPER03'] = ((traindf['DCGODSPN03'])/(traindf['CCGODSPN03']+traindf['CCBADSPN03']+traindf['DCGODSPN03']+traindf['DCBADSPN03'])).replace(np.inf, 0)
traindf['DCBADPER03'] = ((traindf['DCBADSPN03'])/(traindf['CCGODSPN03']+traindf['CCBADSPN03']+traindf['DCGODSPN03']+traindf['DCBADSPN03'])).replace(np.inf, 0)

#Final 02
traindf['CCGODPER02'] = ((traindf['CCGODSPN02'])/(traindf['CCGODSPN02']+traindf['CCBADSPN02']+traindf['DCGODSPN02']+traindf['DCBADSPN02'])).replace(np.inf, 0)
traindf['CCBADPER02'] = ((traindf['CCBADSPN02'])/(traindf['CCGODSPN02']+traindf['CCBADSPN02']+traindf['DCGODSPN02']+traindf['DCBADSPN02'])).replace(np.inf, 0)
traindf['DCGODPER02'] = ((traindf['DCGODSPN02'])/(traindf['CCGODSPN02']+traindf['CCBADSPN02']+traindf['DCGODSPN02']+traindf['DCBADSPN02'])).replace(np.inf, 0)
traindf['DCBADPER02'] = ((traindf['DCBADSPN02'])/(traindf['CCGODSPN02']+traindf['CCBADSPN02']+traindf['DCGODSPN02']+traindf['DCBADSPN02'])).replace(np.inf, 0)

#Final 01
traindf['CCGODPER01'] = ((traindf['CCGODSPN01'])/(traindf['CCGODSPN01']+traindf['CCBADSPN01']+traindf['DCGODSPN01']+traindf['DCBADSPN01'])).replace(np.inf, 0)
traindf['CCBADPER01'] = ((traindf['CCBADSPN01'])/(traindf['CCGODSPN01']+traindf['CCBADSPN01']+traindf['DCGODSPN01']+traindf['DCBADSPN01'])).replace(np.inf, 0)
traindf['DCGODPER01'] = ((traindf['DCGODSPN01'])/(traindf['CCGODSPN01']+traindf['CCBADSPN01']+traindf['DCGODSPN01']+traindf['DCBADSPN01'])).replace(np.inf, 0)
traindf['DCBADPER01'] = ((traindf['DCBADSPN01'])/(traindf['CCGODSPN01']+traindf['CCBADSPN01']+traindf['DCGODSPN01']+traindf['DCBADSPN01'])).replace(np.inf, 0)

In [28]:
# replace missing with 0
traindf['CCGODPER04'] = traindf.CCGODPER04.fillna(0)
traindf['CCBADPER04'] = traindf.CCBADPER04.fillna(0)
traindf['DCGODPER04'] = traindf.DCGODPER04.fillna(0)
traindf['DCBADPER04'] = traindf.DCBADPER04.fillna(0)

traindf['CCGODPER03'] = traindf.CCGODPER03.fillna(0)
traindf['CCBADPER03'] = traindf.CCBADPER03.fillna(0)
traindf['DCGODPER03'] = traindf.DCGODPER03.fillna(0)
traindf['DCBADPER03'] = traindf.DCBADPER03.fillna(0)

traindf['CCGODPER02'] = traindf.CCGODPER02.fillna(0)
traindf['CCBADPER02'] = traindf.CCBADPER02.fillna(0)
traindf['DCGODPER02'] = traindf.DCGODPER02.fillna(0)
traindf['DCBADPER02'] = traindf.DCBADPER02.fillna(0)

traindf['CCGODPER01'] = traindf.CCGODPER01.fillna(0)
traindf['CCBADPER01'] = traindf.CCBADPER01.fillna(0)
traindf['DCGODPER01'] = traindf.DCGODPER01.fillna(0)
traindf['DCBADPER01'] = traindf.DCBADPER01.fillna(0)

In [29]:
del traindf['CCGODSPN04'],traindf['CC_SPEND_MON_04_MED'],traindf['CC_SPEND_MON_04_HMD'],traindf['CC_SPEND_MON_04_ATM']
del traindf['CCBADSPN04'],traindf['CC_SPEND_MON_04_ENT'],traindf['CC_SPEND_MON_04_RST'],traindf['CC_SPEND_MON_04_HTL']
del traindf['CC_SPEND_MON_04_HBY'],traindf['CC_SPEND_MON_04_CARE'],traindf['CC_SPEND_MON_04_TRL'],traindf['CC_SPEND_MON_04_JER']
del traindf['CCGODSPN03'],traindf['CC_SPEND_MON_03_MED'],traindf['CC_SPEND_MON_03_HMD'],traindf['CC_SPEND_MON_03_ATM']
del traindf['CCBADSPN03'],traindf['CC_SPEND_MON_03_ENT'],traindf['CC_SPEND_MON_03_RST'],traindf['CC_SPEND_MON_03_HTL']
del traindf['CC_SPEND_MON_03_HBY'],traindf['CC_SPEND_MON_03_CARE'],traindf['CC_SPEND_MON_03_TRL'],traindf['CC_SPEND_MON_03_JER']
del traindf['CCGODSPN02'],traindf['CC_SPEND_MON_02_MED'],traindf['CC_SPEND_MON_02_HMD'],traindf['CC_SPEND_MON_02_ATM']
del traindf['CCBADSPN02'],traindf['CC_SPEND_MON_02_ENT'],traindf['CC_SPEND_MON_02_RST'],traindf['CC_SPEND_MON_02_HTL']
del traindf['CC_SPEND_MON_02_HBY'],traindf['CC_SPEND_MON_02_CARE'],traindf['CC_SPEND_MON_02_TRL'],traindf['CC_SPEND_MON_02_JER']
del traindf['CCGODSPN01'],traindf['CC_SPEND_MON_01_MED'],traindf['CC_SPEND_MON_01_HMD'],traindf['CC_SPEND_MON_01_ATM']
del traindf['CCBADSPN01'],traindf['CC_SPEND_MON_01_ENT'],traindf['CC_SPEND_MON_01_RST'],traindf['CC_SPEND_MON_01_HTL']
del traindf['CC_SPEND_MON_01_HBY'],traindf['CC_SPEND_MON_01_CARE'],traindf['CC_SPEND_MON_01_TRL'],traindf['CC_SPEND_MON_01_JER']
del traindf['DC_SPEND_MON_05_ENT'],traindf['DC_SPEND_MON_05_MED'],traindf['DC_SPEND_MON_05_HMD'],traindf['DC_SPEND_MON_05_HBY']
del traindf['DC_SPEND_MON_05_CARE'],traindf['DC_SPEND_MON_05_TRL'],traindf['DC_SPEND_MON_05_RST'],traindf['DC_SPEND_MON_05_JER']
del traindf['DC_SPEND_MON_05_HTL'],traindf['DC_SPEND_MON_05_ATM']

In [30]:
del traindf['DCGODSPN04'],traindf['DC_SPEND_MON_04_MED'],traindf['DC_SPEND_MON_04_HMD'],traindf['DC_SPEND_MON_04_ATM']
del traindf['DCBADSPN04'],traindf['DC_SPEND_MON_04_ENT'],traindf['DC_SPEND_MON_04_RST'],traindf['DC_SPEND_MON_04_HTL']
del traindf['DC_SPEND_MON_04_HBY'],traindf['DC_SPEND_MON_04_CARE'],traindf['DC_SPEND_MON_04_TRL'],traindf['DC_SPEND_MON_04_JER']
del traindf['DCGODSPN03'],traindf['DC_SPEND_MON_03_MED'],traindf['DC_SPEND_MON_03_HMD'],traindf['DC_SPEND_MON_03_ATM']
del traindf['DCBADSPN03'],traindf['DC_SPEND_MON_03_ENT'],traindf['DC_SPEND_MON_03_RST'],traindf['DC_SPEND_MON_03_HTL']
del traindf['DC_SPEND_MON_03_HBY'],traindf['DC_SPEND_MON_03_CARE'],traindf['DC_SPEND_MON_03_TRL'],traindf['DC_SPEND_MON_03_JER']
del traindf['DCGODSPN02'],traindf['DC_SPEND_MON_02_MED'],traindf['DC_SPEND_MON_02_HMD'],traindf['DC_SPEND_MON_02_ATM']
del traindf['DCBADSPN02'],traindf['DC_SPEND_MON_02_ENT'],traindf['DC_SPEND_MON_02_RST'],traindf['DC_SPEND_MON_02_HTL']
del traindf['DC_SPEND_MON_02_HBY'],traindf['DC_SPEND_MON_02_CARE'],traindf['DC_SPEND_MON_02_TRL'],traindf['DC_SPEND_MON_02_JER']
del traindf['DCGODSPN01'],traindf['DC_SPEND_MON_01_MED'],traindf['DC_SPEND_MON_01_HMD'],traindf['DC_SPEND_MON_01_ATM']
del traindf['DCBADSPN01'],traindf['DC_SPEND_MON_01_ENT'],traindf['DC_SPEND_MON_01_RST'],traindf['DC_SPEND_MON_01_HTL']
del traindf['DC_SPEND_MON_01_HBY'],traindf['DC_SPEND_MON_01_CARE'],traindf['DC_SPEND_MON_01_TRL'],traindf['DC_SPEND_MON_01_JER']

In [31]:
traindf.shape

(300000, 222)

In [32]:
# diff of salary - EMI
traindf['HLEMIPERC03'] = traindf.fillna(0)['SAL_MON_03']/traindf.fillna(0)['HL_EMI']
traindf['OLEMIPERC03'] = traindf.fillna(0)['SAL_MON_03']/traindf.fillna(0)['SCRUB_EMI']

traindf['HLEMIPERC02'] = traindf.fillna(0)['SAL_MON_02']/traindf.fillna(0)['HL_EMI']
traindf['OLEMIPERC02'] = traindf.fillna(0)['SAL_MON_02']/traindf.fillna(0)['SCRUB_EMI']

traindf['HLEMIPERC01'] = traindf.fillna(0)['SAL_MON_01']/traindf.fillna(0)['HL_EMI']
traindf['OLEMIPERC01'] = traindf.fillna(0)['SAL_MON_01']/traindf.fillna(0)['SCRUB_EMI']

#replace Nan with 0
traindf['HLEMIPERC03'] = traindf.HLEMIPERC03.fillna(0)
traindf['OLEMIPERC03'] = traindf.OLEMIPERC03.fillna(0)
traindf['HLEMIPERC02'] = traindf.HLEMIPERC02.fillna(0)
traindf['OLEMIPERC02'] = traindf.OLEMIPERC02.fillna(0)
traindf['HLEMIPERC01'] = traindf.HLEMIPERC01.fillna(0)
traindf['OLEMIPERC01'] = traindf.OLEMIPERC01.fillna(0)

In [33]:
traindf.shape

(300000, 228)

In [34]:
# Columns to delete
del traindf['ZIP_CODE_FINAL'],traindf['TOP_100_CORP'],traindf['TOP_800_CORP']

In [35]:
# y
traindf['RESPONDERS'] = np.where(traindf['RESPONDERS']=='Y', 1, 0)

In [36]:
# There are now 228 variables in the training set data
nonnumcols = traindf.dtypes[traindf.dtypes == "object"].index
print (len(nonnumcols))
#print (nonnumcols)

33


In [37]:
print ('Before One hit encoding', traindf.shape)

Before One hit encoding (300000, 225)


In [38]:
traindf = pd.get_dummies(traindf, 
                         columns=['ACT_TYPE','GENDER','LEGAL_ENTITY','CC_HOLD','CC_ACTIVE','DC_HOLD','DC_ACTIVE',
                                  'FINANCE_MON_01','JOBS_MON_01','LIFESTYLE_MON_01','INMON_01KET_MON_01',
                                  'TRAVEL_MON_01','CHANNEL_CLICK_DISP','DESIGNATION_FINAL','TOP14_CITY','TOP9_CITY',
                                  'PA_PQ_TAG','NEFT_CC_CATEGORY','NEFT_DC_CATEGORY','TPT_DC_CATEGORY_MON_01',
                                  'TPT_CC_CATEGORY_MON_01','IMPS_CC_CATEGORY_MON_01','CARD_AUTOMOBILE_MON_01',
                                  'CARD_HOTEL_MON_01','CARD_JEWELLERY_MON_01','CARD_RESTAURANT_MON_01',
                                  'CARD_TRAVEL_MON_01','CARD_PRSNL_CARE_MON_01','CARD_HOBBY_MON_01',
                                  'CARD_HOME_DECOR_MON_01','CARD_MEDICAL_MON_01','CARD_ENTMNT_MON_01',
                                  'TOP_CORP_TAG'], drop_first=True)
print ('After One hot encoding',traindf.shape)

After One hot encoding (300000, 470)


In [40]:
#testdf = pd.read_csv('/Users/Vishy/Files/AVDatahack/CrossSell/RawData/test.csv')
testdf = pd.read_csv('/data/test.csv')
print (testdf.shape)
testdf.head()

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,26,28,29,30,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(200000, 332)


,CUSTOMER_ID,ACT_TYPE,GENDER,LEGAL_ENTITY,OCCUP_ALL_NEW,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,BL_TAG,TWL_TAG,PL_TAG,AL_CNC_TAG,AGRI_TAG,CE_TAG,CV_TAG,EDU_TAG,TL_TAG,OTHER_LOANS_TAG,LAP_TAG,LAS_TAG,EEG_TAG,BBG_TAG,HL_TAG,GL_TAG,AL_CLOSED,PL_CLOSED,BL_CLOSED,TWL_CLOSED,CV_CLOSED,LAP_CLOSED,LAS_CLOSED,EEG_CLOSED,BBG_CLOSED,GL_CLOSED,CC_HOLD_MON_03,CC_HOLD_MON_02,CC_HOLD_MON_01,CC_ACTIVE_MON_03,CC_ACTIVE_MON_02,CC_ACTIVE_MON_01,CC_HOLD,CC_ACTIVE,DC_HOLD,DC_ACTIVE_MON_03,DC_ACTIVE_MON_02,DC_ACTIVE_MON_01,DC_ACTIVE,DC_SPEND_MON_03,DC_SPEND_MON_02,DC_SPEND_MON_01,CC_SPEND_MON_04,CC_SPEND_MON_01,CC_SPEND_MON_02,CC_SPEND_MON_03,DC_SPEND_MON_04,PM_FD_MON_01,PM_FD_MON_02,PM_FD_MON_04,CR_LIMIT,STMT_MON_03,STMT_MON_02,STMT_MON_01,COC_ELIGIBLE,LAP_SCRUB_LIVE,LAS_SCRUB_LIVE,PL_SCRUB_LIVE,GL_SCRUB_LIVE,TWL_SCRUB_LIVE,BL_SCRUB_LIVE,CEL_SCRUB_LIVE,TL_SCRUB_LIVE,AL_SCRUB_LIVE,LAP_SCRUB_CLOSED,LAS_SCRUB_CLOSED,PL_SCRUB_CLOSED,GL_SCRUB_CLOSED,TWL_SCRUB_CLOSED,BL_SCRUB_CLOSED,CEL_SCRUB_CLOSED,TL_SCRUB_CLOSED,AL_SCRUB_CLOSED,MATURITY_GL,MATURITY_LAP,MATURITY_LAS,STMT_CON_DAE_ACTIVE_MON_01,BTE_ACTIVE_MON_01,COC_ACTIVE_MON_01,PL2CC_ACTIVE_MON_01,MER_EMI_ACTIVE_MON_01,TRN_CON_DAE_ACTIVE_MON_01,STMT_CON_DAE_CLOSED_MON_01,BTE_CLOSED_MON_01,COC_CLOSED_MON_01,PL2CC_CLOSED_MON_01,MER_EMI_CLOSED_MON_01,TRN_CON_DAE_CLOSED_MON_01,EOP_BAL_MON_01,AMB_MON_04,AMB_MON_01,AMB_MON_02,AMB_MON_03,NB_MON_01_CNT,NB_MON_02_CNT,NB_MON_03_CNT,CDMAT_MON_01,CDMAT_MON_04,MF_MON_04,MF_MON_01,RD_MON_04,RD_MON_01,FD_MON_04,FD_MON_01,LI_MON_04,LI_MON_01,GI_MON_04,GI_MON_01,CC_TXN_MON_01,CC_SPEND_MON_06,CC_TXN_MON_06,CC_TXN_MON_03,CC_TXN_MON_02,CC_TXN_MON_04,CC_SPEND_MON_05,CC_TXN_MON_05,DC_TXN_MON_04,DC_TXN_MON_01,DC_SPEND_MON_06,DC_TXN_MON_06,DC_SPEND_MON_05,DC_TXN_MON_05,DC_TXN_MON_03,DC_TXN_MON_02,DC_SPEND_MON_04_ENT,DC_SPEND_MON_04_MED,DC_SPEND_MON_04_GRC,DC_SPEND_MON_04_HMD,DC_SPEND_MON_04_HBY,DC_SPEND_MON_04_CARE,DC_SPEND_MON_04_TRL,DC_SPEND_MON_04_RST,DC_SPEND_MON_04_JER,DC_SPEND_MON_04_HTL,DC_SPEND_MON_04_ATM,DC_SPEND_MON_03_ENT,DC_SPEND_MON_03_MED,DC_SPEND_MON_03_HMD,DC_SPEND_MON_03_HBY,DC_SPEND_MON_03_CARE,DC_SPEND_MON_03_TRL,DC_SPEND_MON_03_RST,DC_SPEND_MON_03_JER,DC_SPEND_MON_03_HTL,DC_SPEND_MON_03_ATM,DC_SPEND_MON_02_ENT,DC_SPEND_MON_02_MED,DC_SPEND_MON_02_HMD,DC_SPEND_MON_02_HBY,DC_SPEND_MON_02_CARE,DC_SPEND_MON_02_TRL,DC_SPEND_MON_02_RST,DC_SPEND_MON_02_JER,DC_SPEND_MON_02_HTL,DC_SPEND_MON_02_ATM,DC_SPEND_MON_01_ENT,DC_SPEND_MON_01_MED,DC_SPEND_MON_01_HMD,DC_SPEND_MON_01_HBY,DC_SPEND_MON_01_CARE,DC_SPEND_MON_01_TRL,DC_SPEND_MON_01_RST,DC_SPEND_MON_01_JER,DC_SPEND_MON_01_HTL,DC_SPEND_MON_01_ATM,DC_SPEND_MON_05_ENT,DC_SPEND_MON_05_MED,DC_SPEND_MON_05_HMD,DC_SPEND_MON_05_HBY,DC_SPEND_MON_05_CARE,DC_SPEND_MON_05_TRL,DC_SPEND_MON_05_RST,DC_SPEND_MON_05_JER,DC_SPEND_MON_05_HTL,DC_SPEND_MON_05_ATM,CC_SPEND_MON_04_ENT,CC_SPEND_MON_04_MED,CC_SPEND_MON_04_HMD,CC_SPEND_MON_04_HBY,CC_SPEND_MON_04_CARE,CC_SPEND_MON_04_TRL,CC_SPEND_MON_04_RST,CC_SPEND_MON_04_JER,CC_SPEND_MON_04_HTL,CC_SPEND_MON_04_ATM,CC_SPEND_MON_02_ENT,CC_SPEND_MON_02_MED,CC_SPEND_MON_02_HMD,CC_SPEND_MON_02_HBY,CC_SPEND_MON_02_CARE,CC_SPEND_MON_02_TRL,CC_SPEND_MON_02_RST,CC_SPEND_MON_02_JER,CC_SPEND_MON_02_HTL,CC_SPEND_MON_02_ATM,CC_SPEND_MON_01_ENT,CC_SPEND_MON_01_MED,CC_SPEND_MON_01_HMD,CC_SPEND_MON_01_HBY,CC_SPEND_MON_01_CARE,CC_SPEND_MON_01_TRL,CC_SPEND_MON_01_RST,CC_SPEND_MON_01_JER,CC_SPEND_MON_01_HTL,CC_SPEND_MON_01_ATM,FINANCE_MON_01,JOBS_MON_01,LIFESTYLE_MON_01,INMON_01KET_MON_01,TRAVEL_MON_01,CHANNEL_CLICK_DISP,DESIGNATION_FINAL,TOP14_CITY,TOP9_CITY,PL2CC_ELIGIBLE,D_AMT_L3_MON_06,C_AMT_L3_MON_06,C_COUNT_L3_MON_06,D_COUNT_L3_MON_06,MAX_C_AMT_L3_MON_06,D_AMT_L3_MON_05,C_AMT_L3_MON_05,C_COUNT_L3_MON_05,D_COUNT_L3_MON_05,MAX_C_AMT_L3_MON_05,D_AMT_L3_MON_04,C_AMT_L3_MON_04,C_COUNT_L3_MON_04,D_COUNT_L3_MON_04,MAX_C_AMT_L3_MON_04,D_AMT_L3_MON_03,C_AMT_L3_MON_03,C_COUNT_L3_MON_03,D_COUNT_L3_MON_03,MAX_C_AMT_L3_MON_03,D_AMT_L3_MON_02,C_AMT_L3_MON_02,C_COUNT_L3_MON_02,D_COUNT_L3_MON_02,MAX_C_AMT_L3_

In [41]:
# replace Binary Nan as flag 0
testdf['BL_TAG'] = np.where(testdf['BL_TAG']=='Y', 1, 0)
testdf['TWL_TAG'] = np.where(testdf['TWL_TAG']=='Y', 1, 0)
testdf['PL_TAG'] = np.where(testdf['PL_TAG']=='Y', 1, 0)
testdf['AL_CNC_TAG'] = np.where(testdf['AL_CNC_TAG']=='Y', 1, 0)
testdf['AGRI_TAG'] = np.where(testdf['AGRI_TAG']=='Y', 1, 0)
testdf['CE_TAG'] = np.where(testdf['CE_TAG']=='Y', 1, 0)
testdf['CV_TAG'] = np.where(testdf['CV_TAG']=='Y', 1, 0)
testdf['EDU_TAG'] = np.where(testdf['EDU_TAG']=='Y', 1, 0)
testdf['TL_TAG'] = np.where(testdf['TL_TAG']=='Y', 1, 0)
testdf['OTHER_LOANS_TAG'] = np.where(testdf['OTHER_LOANS_TAG']=='Y', 1, 0)
testdf['LAP_TAG'] = np.where(testdf['LAP_TAG']=='Y', 1, 0)
testdf['LAS_TAG'] = np.where(testdf['LAS_TAG']=='Y', 1, 0)
testdf['HL_TAG'] = np.where(testdf['HL_TAG']=='Y', 1, 0)
testdf['GL_TAG'] = np.where(testdf['GL_TAG']=='Y', 1, 0)
testdf['AL_CLOSED'] = np.where(testdf['AL_CLOSED']=='Y', 1, 0)
testdf['PL_CLOSED'] = np.where(testdf['PL_CLOSED']=='Y', 1, 0)
testdf['BL_CLOSED'] = np.where(testdf['BL_CLOSED']=='Y', 1, 0)

In [42]:
# replace Binary Nan as flag 0
testdf['TWL_CLOSED'] = np.where(testdf['TWL_CLOSED']=='Y', 1, 0)
testdf['CV_CLOSED'] = np.where(testdf['CV_CLOSED']=='Y', 1, 0)
testdf['LAP_CLOSED'] = np.where(testdf['LAP_CLOSED']=='Y', 1, 0)
testdf['LAS_CLOSED'] = np.where(testdf['LAS_CLOSED']=='Y', 1, 0)
testdf['GL_CLOSED'] = np.where(testdf['GL_CLOSED']=='Y', 1, 0)
testdf['CC_HOLD_MON_03'] = np.where(testdf['CC_HOLD_MON_03']=='Y', 1, 0)
testdf['CC_HOLD_MON_02'] = np.where(testdf['CC_HOLD_MON_02']=='Y', 1, 0)
testdf['CC_HOLD_MON_01'] = np.where(testdf['CC_HOLD_MON_01']=='Y', 1, 0)
testdf['CC_ACTIVE_MON_03'] = np.where(testdf['CC_ACTIVE_MON_03']=='Y', 1, 0)
testdf['CC_ACTIVE_MON_02'] = np.where(testdf['CC_ACTIVE_MON_02']=='Y', 1, 0)
testdf['CC_ACTIVE_MON_01'] = np.where(testdf['CC_ACTIVE_MON_01']=='Y', 1, 0)
testdf['DC_ACTIVE_MON_03'] = np.where(testdf['DC_ACTIVE_MON_03']=='Y', 1, 0)
testdf['DC_ACTIVE_MON_02'] = np.where(testdf['DC_ACTIVE_MON_02']=='Y', 1, 0)
testdf['DC_ACTIVE_MON_01'] = np.where(testdf['DC_ACTIVE_MON_01']=='Y', 1, 0)
testdf['BTE_ACTIVE_MON_01'] = np.where(testdf['BTE_ACTIVE_MON_01']=='Y', 1, 0)
testdf['COC_ACTIVE_MON_01'] = np.where(testdf['COC_ACTIVE_MON_01']=='Y', 1, 0)
testdf['PL2CC_ACTIVE_MON_01'] = np.where(testdf['PL2CC_ACTIVE_MON_01']=='Y', 1, 0)
testdf['MER_EMI_ACTIVE_MON_01'] = np.where(testdf['MER_EMI_ACTIVE_MON_01']=='Y', 1, 0)

In [43]:
# replace Binary Nan as flag 0
testdf['TRN_CON_DAE_ACTIVE_MON_01'] = np.where(testdf['TRN_CON_DAE_ACTIVE_MON_01']=='Y', 1, 0)
testdf['BTE_CLOSED_MON_01'] = np.where(testdf['BTE_CLOSED_MON_01']=='Y', 1, 0)
testdf['COC_CLOSED_MON_01'] = np.where(testdf['COC_CLOSED_MON_01']=='Y', 1, 0)
testdf['PL2CC_CLOSED_MON_01'] = np.where(testdf['PL2CC_CLOSED_MON_01']=='Y', 1, 0)
testdf['TRN_CON_DAE_CLOSED_MON_01'] = np.where(testdf['TRN_CON_DAE_CLOSED_MON_01']=='Y', 1, 0)
testdf['LAST_3MTHS_INQ'] = np.where(testdf['LAST_3MTHS_INQ']=='Y', 1, 0)
testdf['PREV_3MTHS_INQ'] = np.where(testdf['PREV_3MTHS_INQ']=='Y', 1, 0)
testdf['CREDIT_ACT'] = np.where(testdf['CREDIT_ACT']=='Y', 1, 0)
testdf['DEBIT_ACT'] = np.where(testdf['DEBIT_ACT']=='Y', 1, 0)
testdf['CEL_TAG'] = np.where(testdf['CEL_TAG']=='Y', 1, 0)
testdf['EL_TAG'] = np.where(testdf['EL_TAG']=='Y', 1, 0)
testdf['AGL_TAG'] = np.where(testdf['AGL_TAG']=='Y', 1, 0)
testdf['CVL_TAG'] = np.where(testdf['CVL_TAG']=='Y', 1, 0)
testdf['AL_TAG'] = np.where(testdf['AL_TAG']=='Y', 1, 0)

In [44]:
# convert non 1 to 0
testdf['LAP_SCRUB_LIVE'] = np.where(testdf['LAP_SCRUB_LIVE']==1.0, 1, 0)
testdf['LAS_SCRUB_LIVE'] = np.where(testdf['LAS_SCRUB_LIVE']==1.0, 1, 0)
testdf['GL_SCRUB_LIVE'] = np.where(testdf['GL_SCRUB_LIVE']==1.0, 1, 0)
testdf['TWL_SCRUB_LIVE'] = np.where(testdf['TWL_SCRUB_LIVE']==1.0, 1, 0)
testdf['CEL_SCRUB_LIVE'] = np.where(testdf['CEL_SCRUB_LIVE']==1.0, 1, 0)
testdf['TL_SCRUB_LIVE'] = np.where(testdf['TL_SCRUB_LIVE']==1.0, 1, 0)
testdf['AL_SCRUB_LIVE'] = np.where(testdf['AL_SCRUB_LIVE']==1.0, 1, 0)
testdf['PL_SCRUB_LIVE'] = np.where(testdf['PL_SCRUB_LIVE']==1.0, 1, 0)
testdf['BL_SCRUB_LIVE'] = np.where(testdf['BL_SCRUB_LIVE']==1.0, 1, 0)
testdf['LAP_SCRUB_CLOSED'] = np.where(testdf['LAP_SCRUB_CLOSED']==1.0, 1, 0)
testdf['LAS_SCRUB_CLOSED'] = np.where(testdf['LAS_SCRUB_CLOSED']==1.0, 1, 0)
testdf['GL_SCRUB_CLOSED'] = np.where(testdf['GL_SCRUB_CLOSED']==1.0, 1, 0)
testdf['TWL_SCRUB_CLOSED'] = np.where(testdf['TWL_SCRUB_CLOSED']==1.0, 1, 0)
testdf['CEL_SCRUB_CLOSED'] = np.where(testdf['CEL_SCRUB_CLOSED']==1.0, 1, 0)
testdf['TL_SCRUB_CLOSED'] = np.where(testdf['TL_SCRUB_CLOSED']==1.0, 1, 0)
testdf['AL_SCRUB_CLOSED'] = np.where(testdf['AL_SCRUB_CLOSED']==1.0, 1, 0)
testdf['PL_SCRUB_CLOSED'] = np.where(testdf['PL_SCRUB_CLOSED']==1.0, 1, 0)
testdf['BL_SCRUB_CLOSED'] = np.where(testdf['BL_SCRUB_CLOSED']==1.0, 1, 0)

In [45]:
# Non useful features
del testdf['OCCUP_ALL_NEW'],testdf['EEG_TAG'],testdf['EEG_CLOSED']

In [46]:
testdf.shape

(200000, 329)

In [47]:
# Identify if it is BUSINESS OR AGRI BUSINESS MAN
testdf['NONBUSNONAGRI'] = np.where((testdf['BL_TAG']+testdf['AGRI_TAG']+testdf['CE_TAG']+testdf['CV_TAG']+
                                     testdf['TL_TAG']+testdf['BBG_TAG']+testdf['BL_SCRUB_LIVE']+
                                     testdf['CEL_SCRUB_LIVE']+testdf['TL_SCRUB_LIVE'])>= 1,1,0)

In [48]:
# New variables related to Loan to Eliminate many varaibles by taking the count
# All variables starting with GB is BANK OF GARDENIA are within the bank
testdf['NONBUSNONAGRI'] = np.where((testdf['BL_TAG']+testdf['AGRI_TAG']+testdf['CE_TAG']+testdf['CV_TAG']+
                                    testdf['TL_TAG']+testdf['BBG_TAG']+testdf['BL_SCRUB_LIVE']+
                                    testdf['CEL_SCRUB_LIVE']+testdf['TL_SCRUB_LIVE'])>= 1,1,0)
testdf['GBSCLN'] = (testdf['TWL_TAG']+testdf['AL_CNC_TAG']+testdf['CE_TAG']+testdf['CV_TAG']+
                    testdf['TL_TAG']+testdf['LAP_TAG']+testdf['LAS_TAG']+testdf['BBG_TAG']+testdf['HL_TAG']+testdf['GL_TAG'])
testdf['GBUNSCLN'] = (testdf['BL_TAG']+testdf['PL_TAG']+testdf['AGRI_TAG']+testdf['EDU_TAG']+testdf['OTHER_LOANS_TAG'])
testdf['GBPLLN'] = (testdf['PL_TAG'])
testdf['GBSCCLSDLN'] = (testdf['AL_CLOSED']+testdf['TWL_CLOSED']+testdf['CV_CLOSED']+testdf['LAP_CLOSED']+
                        testdf['LAS_CLOSED']+testdf['BBG_CLOSED']+testdf['GL_CLOSED'])
testdf['GBUNSCCLSDLN'] = (testdf['PL_CLOSED']+testdf['BL_CLOSED'])
testdf['GBPLLNCLSD'] = (testdf['PL_CLOSED'])
# All variables starting with OB are other Bank details

testdf['OBSCLN'] = (testdf['LAP_SCRUB_LIVE']+testdf['LAS_SCRUB_LIVE']+testdf['GL_SCRUB_LIVE']+
                    testdf['TWL_SCRUB_LIVE']+testdf['CEL_SCRUB_LIVE']+testdf['TL_SCRUB_LIVE']+testdf['AL_SCRUB_LIVE'])
testdf['OBUNSCLN'] = (testdf['PL_SCRUB_LIVE']+testdf['BL_SCRUB_LIVE'])
testdf['OBBPLLN'] = (testdf['PL_SCRUB_LIVE'])
testdf['OBSCLNCLSD'] = (testdf['LAP_SCRUB_CLOSED']+testdf['LAS_SCRUB_CLOSED']+testdf['GL_SCRUB_CLOSED']+
                        testdf['TWL_SCRUB_CLOSED']+testdf['CEL_SCRUB_CLOSED']+testdf['TL_SCRUB_CLOSED']+
                        testdf['AL_SCRUB_CLOSED'])
testdf['OBUNSCLNCLSD'] = (testdf['PL_SCRUB_CLOSED']+testdf['BL_SCRUB_CLOSED'])
testdf['OBPLLNCLSD'] = (testdf['PL_SCRUB_CLOSED'])

In [49]:
# Delete the Unwanted variables now
del testdf['BL_TAG'],testdf['TWL_TAG'],testdf['PL_TAG'],testdf['AL_CNC_TAG'],testdf['AGRI_TAG']
del testdf['CE_TAG'],testdf['CV_TAG'],testdf['EDU_TAG'],testdf['TL_TAG'],testdf['OTHER_LOANS_TAG']
del testdf['LAP_TAG'],testdf['LAS_TAG'],testdf['BBG_TAG'],testdf['HL_TAG'],testdf['GL_TAG']
del testdf['AL_CLOSED'],testdf['PL_CLOSED'],testdf['BL_CLOSED'],testdf['TWL_CLOSED']
del testdf['CV_CLOSED'],testdf['LAP_CLOSED'],testdf['LAS_CLOSED'],testdf['BBG_CLOSED'],testdf['GL_CLOSED']
del testdf['LAP_SCRUB_LIVE'],testdf['LAS_SCRUB_LIVE'],testdf['GL_SCRUB_LIVE'],testdf['TWL_SCRUB_LIVE']
del testdf['CEL_SCRUB_LIVE'],testdf['TL_SCRUB_LIVE'],testdf['AL_SCRUB_LIVE'],testdf['PL_SCRUB_LIVE']
del testdf['BL_SCRUB_LIVE'],testdf['LAP_SCRUB_CLOSED'],testdf['LAS_SCRUB_CLOSED'],testdf['GL_SCRUB_CLOSED']
del testdf['TWL_SCRUB_CLOSED'],testdf['CEL_SCRUB_CLOSED'],testdf['TL_SCRUB_CLOSED'],testdf['AL_SCRUB_CLOSED']
del testdf['PL_SCRUB_CLOSED'],testdf['BL_SCRUB_CLOSED']

In [50]:
testdf.shape

(200000, 300)

In [51]:
# MATURITY_GL, MATURITY_LAP, MATURITY_LAS, CLOSED_DATE
testdf['TODAY'] = time.strftime("%d%b%Y")
testdf['MGLDAYS'] = pd.to_datetime(testdf['MATURITY_GL'],format='%d%b%Y' )-pd.to_datetime(testdf['TODAY'], format='%d%b%Y')
testdf['MLAPDAYS'] = pd.to_datetime(testdf['MATURITY_LAP'],format='%d%b%Y' )-pd.to_datetime(testdf['TODAY'], format='%d%b%Y')
testdf['MLASDAYS'] = pd.to_datetime(testdf['MATURITY_LAS'],format='%d%b%Y' )-pd.to_datetime(testdf['TODAY'], format='%d%b%Y')
testdf['LLOANDAYS'] = pd.to_datetime(testdf['CLOSED_DATE'],format='%d%b%Y' )-pd.to_datetime(testdf['TODAY'], format='%d%b%Y')

In [52]:
# Convert the days into Numbers
testdf['MGLDAYS'] = testdf['MGLDAYS'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)
testdf['MLAPDAYS'] = testdf['MLAPDAYS'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)
testdf['MLASDAYS'] = testdf['MLASDAYS'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)
testdf['LLOANDAYS'] = testdf['LLOANDAYS'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days)

In [53]:
# Everything less than 0 should be made 0
testdf['MGLDAYS'] = np.where(testdf['MGLDAYS'] <= 0,0,testdf.MGLDAYS)
testdf['MLAPDAYS'] = np.where(testdf['MLAPDAYS'] <= 0,0,testdf.MLAPDAYS)
testdf['MLASDAYS'] = np.where(testdf['MLASDAYS'] <= 0,0,testdf.MLASDAYS)
testdf['LLOANDAYS'] = np.where(testdf['LLOANDAYS'] <= 0,0,testdf.LLOANDAYS)

In [54]:
# replace all NaN with 0
testdf.MGLDAYS.fillna(0, inplace=True)
testdf.MLAPDAYS.fillna(0, inplace=True)
testdf.MLASDAYS.fillna(0, inplace=True)
testdf.LLOANDAYS.fillna(0, inplace=True)

In [55]:
del testdf['MATURITY_GL'],testdf['MATURITY_LAP'],testdf['MATURITY_LAS'],testdf['CLOSED_DATE'],testdf['TODAY']

In [56]:
testdf.shape

(200000, 300)

In [57]:
#Group Invest snd insurance
testdf['TOTINVEST04'] = testdf.fillna(0)['CDMAT_MON_04']+testdf.fillna(0)['MF_MON_04']+testdf.fillna(0)['RD_MON_04']+testdf.fillna(0)['FD_MON_04']
testdf['TOTINVEST01'] = testdf.fillna(0)['CDMAT_MON_01']+testdf.fillna(0)['MF_MON_01']+testdf.fillna(0)['RD_MON_01']+testdf.fillna(0)['FD_MON_01']
testdf['TOTINS04'] = testdf.fillna(0)['LI_MON_04']+testdf.fillna(0)['GI_MON_04']
testdf['TOTINS01'] = testdf.fillna(0)['LI_MON_01']+testdf.fillna(0)['GI_MON_01']

del testdf['MF_MON_04'],testdf['RD_MON_04'],testdf['FD_MON_04'],testdf['MF_MON_01'],testdf['RD_MON_01'],testdf['CDMAT_MON_04']
del testdf['FD_MON_01'],testdf['LI_MON_04'],testdf['GI_MON_04'],testdf['LI_MON_01'],testdf['GI_MON_01'],testdf['CDMAT_MON_01']
testdf.shape

(200000, 292)

In [61]:
# Credit Card mapping

#testdf['CCYESACT03'] = testdf.apply(lambda x: 1 if x['CC_HOLD_MON_03'] == x['CC_ACTIVE_MON_03'] else 0, axis=1)
#testdf['CCYESACT02'] = testdf.apply(lambda x: 1 if x['CC_HOLD_MON_02'] == x['CC_ACTIVE_MON_02'] else 0, axis=1)
#testdf['CCYESACT01'] = testdf.apply(lambda x: 1 if x['CC_HOLD_MON_01'] == x['CC_ACTIVE_MON_01'] else 0, axis=1)

del testdf['CC_HOLD_MON_03'],testdf['CC_ACTIVE_MON_03'],testdf['CC_HOLD_MON_02'],testdf['CC_ACTIVE_MON_02']
del testdf['CC_HOLD_MON_01'],testdf['CC_ACTIVE_MON_01']
testdf.shape

(200000, 289)

In [62]:
# Credit vs Debit spend
testdf['DBVSCR06'] = np.where((testdf.fillna(0)['CC_SPEND_MON_06']-testdf.fillna(0)['DC_SPEND_MON_06'])>=0,1,0)
testdf['DBVSCR05'] = np.where((testdf.fillna(0)['CC_SPEND_MON_05']-testdf.fillna(0)['DC_SPEND_MON_05'])>=0,1,0)
testdf['DBVSCR04'] = np.where((testdf.fillna(0)['CC_SPEND_MON_04']-testdf.fillna(0)['DC_SPEND_MON_04'])>=0,1,0)
testdf['DBVSCR03'] = np.where((testdf.fillna(0)['CC_SPEND_MON_03']-testdf.fillna(0)['DC_SPEND_MON_03'])>=0,1,0)
testdf['DBVSCR02'] = np.where((testdf.fillna(0)['CC_SPEND_MON_02']-testdf.fillna(0)['DC_SPEND_MON_02'])>=0,1,0)
testdf['DBVSCR01'] = np.where((testdf.fillna(0)['CC_SPEND_MON_01']-testdf.fillna(0)['DC_SPEND_MON_01'])>=0,1,0)

# Count of credit higher than debit
testdf['CONTCCDC'] = testdf['DBVSCR06']+testdf['DBVSCR05']+testdf['DBVSCR04']+testdf['DBVSCR03']+testdf['DBVSCR02']+testdf['DBVSCR01']
del testdf['DBVSCR06'],testdf['DBVSCR05'],testdf['DBVSCR04'],testdf['DBVSCR03'],testdf['DBVSCR02'],testdf['DBVSCR01']

# Credit vs Salary Spend
testdf['SAVSCR03'] = np.where((testdf.fillna(0)['CC_SPEND_MON_03']-testdf.fillna(0)['SAL_MON_03'])>=0,1,0)
testdf['SAVSCR02'] = np.where((testdf.fillna(0)['CC_SPEND_MON_02']-testdf.fillna(0)['SAL_MON_02'])>=0,1,0)
testdf['SAVSCR01'] = np.where((testdf.fillna(0)['CC_SPEND_MON_01']-testdf.fillna(0)['SAL_MON_01'])>=0,1,0)

# Count of Credit more than Salary
testdf['CONTCCSL'] = testdf['SAVSCR03']+testdf['SAVSCR02']+testdf['SAVSCR01']
del testdf['SAVSCR03'],testdf['SAVSCR02'],testdf['SAVSCR01']

# DEBIT AMOUNT VS CREDIT AMOUNT
testdf['DBMINCR06'] = np.where((testdf.fillna(0)['D_AMT_L3_MON_06']-testdf.fillna(0)['C_AMT_L3_MON_06'])>=0,1,0)
testdf['DBMINCR05'] = np.where((testdf.fillna(0)['D_AMT_L3_MON_05']-testdf.fillna(0)['C_AMT_L3_MON_05'])>=0,1,0)
testdf['DBMINCR04'] = np.where((testdf.fillna(0)['D_AMT_L3_MON_04']-testdf.fillna(0)['C_AMT_L3_MON_04'])>=0,1,0)
testdf['DBMINCR03'] = np.where((testdf.fillna(0)['D_AMT_L3_MON_03']-testdf.fillna(0)['C_AMT_L3_MON_03'])>=0,1,0)
testdf['DBMINCR02'] = np.where((testdf.fillna(0)['D_AMT_L3_MON_02']-testdf.fillna(0)['C_AMT_L3_MON_02'])>=0,1,0)
testdf['DBMINCR01'] = np.where((testdf.fillna(0)['D_AMT_L3_MON_01']-testdf.fillna(0)['C_AMT_L3_MON_01'])>=0,1,0)

# Count of Credit higher than Debit
testdf['DEBMORCRE'] = testdf['DBMINCR06']+testdf['DBMINCR05']+testdf['DBMINCR04']+testdf['DBMINCR03']+testdf['DBMINCR02']+testdf['DBMINCR01']
del testdf['DBMINCR06'],testdf['DBMINCR05'],testdf['DBMINCR04'],testdf['DBMINCR03'],testdf['DBMINCR02'],testdf['DBMINCR01']

# Check shape
testdf.shape

(200000, 292)

In [63]:
# % Spend Against Credit Limit
testdf['CRLIM03'] = testdf.fillna(0)['STMT_MON_03']/testdf.fillna(0)['CR_LIMIT']
testdf['CRLIM02'] = testdf.fillna(0)['STMT_MON_02']/testdf.fillna(0)['CR_LIMIT']
testdf['CRLIM01'] = testdf.fillna(0)['STMT_MON_01']/testdf.fillna(0)['CR_LIMIT']
testdf['CRLIM03'] = testdf.CRLIM03.fillna(0)
testdf['CRLIM02'] = testdf.CRLIM02.fillna(0)
testdf['CRLIM01'] = testdf.CRLIM01.fillna(0)
testdf.shape

(200000, 295)

In [64]:
# CC & DC Spend Categorize
#CC SPEND

testdf['CCGODSPN04'] = testdf.fillna(0)['CC_SPEND_MON_04_MED']+testdf.fillna(0)['CC_SPEND_MON_04_HMD']+testdf.fillna(0)['CC_SPEND_MON_04_ATM']
testdf['CCBADSPN04'] = (testdf.fillna(0)['CC_SPEND_MON_04_ENT']+testdf.fillna(0)['CC_SPEND_MON_04_RST']+testdf.fillna(0)['CC_SPEND_MON_04_HTL']+
                        testdf.fillna(0)['CC_SPEND_MON_04_HBY']+testdf.fillna(0)['CC_SPEND_MON_04_CARE']+testdf.fillna(0)['CC_SPEND_MON_04_TRL']+
                        testdf.fillna(0)['CC_SPEND_MON_04_JER'])

testdf['CCGODSPN03'] = testdf.fillna(0)['CC_SPEND_MON_03_MED']+testdf.fillna(0)['CC_SPEND_MON_03_HMD']+testdf.fillna(0)['CC_SPEND_MON_03_ATM']
testdf['CCBADSPN03'] = (testdf.fillna(0)['CC_SPEND_MON_03_ENT']+testdf.fillna(0)['CC_SPEND_MON_03_RST']+testdf.fillna(0)['CC_SPEND_MON_03_HTL']+
                        testdf.fillna(0)['CC_SPEND_MON_03_HBY']+testdf.fillna(0)['CC_SPEND_MON_03_CARE']+testdf.fillna(0)['CC_SPEND_MON_03_TRL']+
                        testdf.fillna(0)['CC_SPEND_MON_03_JER'])

testdf['CCGODSPN02'] = testdf.fillna(0)['CC_SPEND_MON_02_MED']+testdf.fillna(0)['CC_SPEND_MON_02_HMD']+testdf.fillna(0)['CC_SPEND_MON_02_ATM']
testdf['CCBADSPN02'] = (testdf.fillna(0)['CC_SPEND_MON_02_ENT']+testdf.fillna(0)['CC_SPEND_MON_02_RST']+testdf.fillna(0)['CC_SPEND_MON_02_HTL']+
                        testdf.fillna(0)['CC_SPEND_MON_02_HBY']+testdf.fillna(0)['CC_SPEND_MON_02_CARE']+testdf.fillna(0)['CC_SPEND_MON_02_TRL']+
                        testdf.fillna(0)['CC_SPEND_MON_02_JER'])

testdf['CCGODSPN01'] = testdf.fillna(0)['CC_SPEND_MON_01_MED']+testdf.fillna(0)['CC_SPEND_MON_01_HMD']+testdf.fillna(0)['CC_SPEND_MON_01_ATM']
testdf['CCBADSPN01'] = (testdf.fillna(0)['CC_SPEND_MON_01_ENT']+testdf.fillna(0)['CC_SPEND_MON_01_RST']+testdf.fillna(0)['CC_SPEND_MON_01_HTL']+
                        testdf.fillna(0)['CC_SPEND_MON_01_HBY']+testdf.fillna(0)['CC_SPEND_MON_01_CARE']+testdf.fillna(0)['CC_SPEND_MON_01_TRL']+
                        testdf.fillna(0)['CC_SPEND_MON_01_JER'])
# DC SPEND

testdf['DCGODSPN04'] = testdf.fillna(0)['DC_SPEND_MON_04_MED']+testdf.fillna(0)['DC_SPEND_MON_04_HMD']+testdf.fillna(0)['DC_SPEND_MON_04_ATM']
testdf['DCBADSPN04'] = (testdf.fillna(0)['DC_SPEND_MON_04_ENT']+testdf.fillna(0)['DC_SPEND_MON_04_RST']+testdf.fillna(0)['DC_SPEND_MON_04_HTL']+
                        testdf.fillna(0)['DC_SPEND_MON_04_HBY']+testdf.fillna(0)['DC_SPEND_MON_04_CARE']+testdf.fillna(0)['DC_SPEND_MON_04_TRL']+
                        testdf.fillna(0)['DC_SPEND_MON_04_JER'])

testdf['DCGODSPN03'] = testdf.fillna(0)['DC_SPEND_MON_03_MED']+testdf.fillna(0)['DC_SPEND_MON_03_HMD']+testdf.fillna(0)['DC_SPEND_MON_03_ATM']
testdf['DCBADSPN03'] = (testdf.fillna(0)['DC_SPEND_MON_03_ENT']+testdf.fillna(0)['DC_SPEND_MON_03_RST']+testdf.fillna(0)['DC_SPEND_MON_03_HTL']+
                        testdf.fillna(0)['DC_SPEND_MON_03_HBY']+testdf.fillna(0)['DC_SPEND_MON_03_CARE']+testdf.fillna(0)['DC_SPEND_MON_03_TRL']+
                        testdf.fillna(0)['DC_SPEND_MON_03_JER'])

testdf['DCGODSPN02'] = testdf.fillna(0)['DC_SPEND_MON_02_MED']+testdf.fillna(0)['DC_SPEND_MON_02_HMD']+testdf.fillna(0)['DC_SPEND_MON_02_ATM']
testdf['DCBADSPN02'] = (testdf.fillna(0)['DC_SPEND_MON_02_ENT']+testdf.fillna(0)['DC_SPEND_MON_02_RST']+testdf.fillna(0)['DC_SPEND_MON_02_HTL']+
                        testdf.fillna(0)['DC_SPEND_MON_02_HBY']+testdf.fillna(0)['DC_SPEND_MON_02_CARE']+testdf.fillna(0)['DC_SPEND_MON_02_TRL']+
                        testdf.fillna(0)['DC_SPEND_MON_02_JER'])

testdf['DCGODSPN01'] = testdf.fillna(0)['DC_SPEND_MON_01_MED']+testdf.fillna(0)['DC_SPEND_MON_01_HMD']+testdf.fillna(0)['DC_SPEND_MON_01_ATM']
testdf['DCBADSPN01'] = (testdf.fillna(0)['DC_SPEND_MON_01_ENT']+testdf.fillna(0)['DC_SPEND_MON_01_RST']+testdf.fillna(0)['DC_SPEND_MON_01_HTL']+
                        testdf.fillna(0)['DC_SPEND_MON_01_HBY']+testdf.fillna(0)['DC_SPEND_MON_01_CARE']+testdf.fillna(0)['DC_SPEND_MON_01_TRL']+
                        testdf.fillna(0)['DC_SPEND_MON_01_JER'])

In [65]:
#Final 04
testdf['CCGODPER04'] = ((testdf['CCGODSPN04'])/(testdf['CCGODSPN04']+testdf['CCBADSPN04']+testdf['DCGODSPN04']+testdf['DCBADSPN04'])).replace(np.inf, 0)
testdf['CCBADPER04'] = ((testdf['CCBADSPN04'])/(testdf['CCGODSPN04']+testdf['CCBADSPN04']+testdf['DCGODSPN04']+testdf['DCBADSPN04'])).replace(np.inf, 0)
testdf['DCGODPER04'] = ((testdf['DCGODSPN04'])/(testdf['CCGODSPN04']+testdf['CCBADSPN04']+testdf['DCGODSPN04']+testdf['DCBADSPN04'])).replace(np.inf, 0)
testdf['DCBADPER04'] = ((testdf['DCBADSPN04'])/(testdf['CCGODSPN04']+testdf['CCBADSPN04']+testdf['DCGODSPN04']+testdf['DCBADSPN04'])).replace(np.inf, 0)

#Final 03
testdf['CCGODPER03'] = ((testdf['CCGODSPN03'])/(testdf['CCGODSPN03']+testdf['CCBADSPN03']+testdf['DCGODSPN03']+testdf['DCBADSPN03'])).replace(np.inf, 0)
testdf['CCBADPER03'] = ((testdf['CCBADSPN03'])/(testdf['CCGODSPN03']+testdf['CCBADSPN03']+testdf['DCGODSPN03']+testdf['DCBADSPN03'])).replace(np.inf, 0)
testdf['DCGODPER03'] = ((testdf['DCGODSPN03'])/(testdf['CCGODSPN03']+testdf['CCBADSPN03']+testdf['DCGODSPN03']+testdf['DCBADSPN03'])).replace(np.inf, 0)
testdf['DCBADPER03'] = ((testdf['DCBADSPN03'])/(testdf['CCGODSPN03']+testdf['CCBADSPN03']+testdf['DCGODSPN03']+testdf['DCBADSPN03'])).replace(np.inf, 0)

#Final 02
testdf['CCGODPER02'] = ((testdf['CCGODSPN02'])/(testdf['CCGODSPN02']+testdf['CCBADSPN02']+testdf['DCGODSPN02']+testdf['DCBADSPN02'])).replace(np.inf, 0)
testdf['CCBADPER02'] = ((testdf['CCBADSPN02'])/(testdf['CCGODSPN02']+testdf['CCBADSPN02']+testdf['DCGODSPN02']+testdf['DCBADSPN02'])).replace(np.inf, 0)
testdf['DCGODPER02'] = ((testdf['DCGODSPN02'])/(testdf['CCGODSPN02']+testdf['CCBADSPN02']+testdf['DCGODSPN02']+testdf['DCBADSPN02'])).replace(np.inf, 0)
testdf['DCBADPER02'] = ((testdf['DCBADSPN02'])/(testdf['CCGODSPN02']+testdf['CCBADSPN02']+testdf['DCGODSPN02']+testdf['DCBADSPN02'])).replace(np.inf, 0)

#Final 01
testdf['CCGODPER01'] = ((testdf['CCGODSPN01'])/(testdf['CCGODSPN01']+testdf['CCBADSPN01']+testdf['DCGODSPN01']+testdf['DCBADSPN01'])).replace(np.inf, 0)
testdf['CCBADPER01'] = ((testdf['CCBADSPN01'])/(testdf['CCGODSPN01']+testdf['CCBADSPN01']+testdf['DCGODSPN01']+testdf['DCBADSPN01'])).replace(np.inf, 0)
testdf['DCGODPER01'] = ((testdf['DCGODSPN01'])/(testdf['CCGODSPN01']+testdf['CCBADSPN01']+testdf['DCGODSPN01']+testdf['DCBADSPN01'])).replace(np.inf, 0)
testdf['DCBADPER01'] = ((testdf['DCBADSPN01'])/(testdf['CCGODSPN01']+testdf['CCBADSPN01']+testdf['DCGODSPN01']+testdf['DCBADSPN01'])).replace(np.inf, 0)

In [66]:
# replace missing with 0
testdf['CCGODPER04'] = testdf.CCGODPER04.fillna(0)
testdf['CCBADPER04'] = testdf.CCBADPER04.fillna(0)
testdf['DCGODPER04'] = testdf.DCGODPER04.fillna(0)
testdf['DCBADPER04'] = testdf.DCBADPER04.fillna(0)

testdf['CCGODPER03'] = testdf.CCGODPER03.fillna(0)
testdf['CCBADPER03'] = testdf.CCBADPER03.fillna(0)
testdf['DCGODPER03'] = testdf.DCGODPER03.fillna(0)
testdf['DCBADPER03'] = testdf.DCBADPER03.fillna(0)

testdf['CCGODPER02'] = testdf.CCGODPER02.fillna(0)
testdf['CCBADPER02'] = testdf.CCBADPER02.fillna(0)
testdf['DCGODPER02'] = testdf.DCGODPER02.fillna(0)
testdf['DCBADPER02'] = testdf.DCBADPER02.fillna(0)

testdf['CCGODPER01'] = testdf.CCGODPER01.fillna(0)
testdf['CCBADPER01'] = testdf.CCBADPER01.fillna(0)
testdf['DCGODPER01'] = testdf.DCGODPER01.fillna(0)
testdf['DCBADPER01'] = testdf.DCBADPER01.fillna(0)

In [67]:
del testdf['CCGODSPN04'],testdf['CC_SPEND_MON_04_MED'],testdf['CC_SPEND_MON_04_HMD'],testdf['CC_SPEND_MON_04_ATM']
del testdf['CCBADSPN04'],testdf['CC_SPEND_MON_04_ENT'],testdf['CC_SPEND_MON_04_RST'],testdf['CC_SPEND_MON_04_HTL']
del testdf['CC_SPEND_MON_04_HBY'],testdf['CC_SPEND_MON_04_CARE'],testdf['CC_SPEND_MON_04_TRL'],testdf['CC_SPEND_MON_04_JER']
del testdf['CCGODSPN03'],testdf['CC_SPEND_MON_03_MED'],testdf['CC_SPEND_MON_03_HMD'],testdf['CC_SPEND_MON_03_ATM']
del testdf['CCBADSPN03'],testdf['CC_SPEND_MON_03_ENT'],testdf['CC_SPEND_MON_03_RST'],testdf['CC_SPEND_MON_03_HTL']
del testdf['CC_SPEND_MON_03_HBY'],testdf['CC_SPEND_MON_03_CARE'],testdf['CC_SPEND_MON_03_TRL'],testdf['CC_SPEND_MON_03_JER']
del testdf['CCGODSPN02'],testdf['CC_SPEND_MON_02_MED'],testdf['CC_SPEND_MON_02_HMD'],testdf['CC_SPEND_MON_02_ATM']
del testdf['CCBADSPN02'],testdf['CC_SPEND_MON_02_ENT'],testdf['CC_SPEND_MON_02_RST'],testdf['CC_SPEND_MON_02_HTL']
del testdf['CC_SPEND_MON_02_HBY'],testdf['CC_SPEND_MON_02_CARE'],testdf['CC_SPEND_MON_02_TRL'],testdf['CC_SPEND_MON_02_JER']
del testdf['CCGODSPN01'],testdf['CC_SPEND_MON_01_MED'],testdf['CC_SPEND_MON_01_HMD'],testdf['CC_SPEND_MON_01_ATM']
del testdf['CCBADSPN01'],testdf['CC_SPEND_MON_01_ENT'],testdf['CC_SPEND_MON_01_RST'],testdf['CC_SPEND_MON_01_HTL']
del testdf['CC_SPEND_MON_01_HBY'],testdf['CC_SPEND_MON_01_CARE'],testdf['CC_SPEND_MON_01_TRL'],testdf['CC_SPEND_MON_01_JER']
del testdf['DC_SPEND_MON_05_ENT'],testdf['DC_SPEND_MON_05_MED'],testdf['DC_SPEND_MON_05_HMD'],testdf['DC_SPEND_MON_05_HBY']
del testdf['DC_SPEND_MON_05_CARE'],testdf['DC_SPEND_MON_05_TRL'],testdf['DC_SPEND_MON_05_RST'],testdf['DC_SPEND_MON_05_JER']
del testdf['DC_SPEND_MON_05_HTL'],testdf['DC_SPEND_MON_05_ATM']

In [68]:
del testdf['DCGODSPN04'],testdf['DC_SPEND_MON_04_MED'],testdf['DC_SPEND_MON_04_HMD'],testdf['DC_SPEND_MON_04_ATM']
del testdf['DCBADSPN04'],testdf['DC_SPEND_MON_04_ENT'],testdf['DC_SPEND_MON_04_RST'],testdf['DC_SPEND_MON_04_HTL']
del testdf['DC_SPEND_MON_04_HBY'],testdf['DC_SPEND_MON_04_CARE'],testdf['DC_SPEND_MON_04_TRL'],testdf['DC_SPEND_MON_04_JER']
del testdf['DCGODSPN03'],testdf['DC_SPEND_MON_03_MED'],testdf['DC_SPEND_MON_03_HMD'],testdf['DC_SPEND_MON_03_ATM']
del testdf['DCBADSPN03'],testdf['DC_SPEND_MON_03_ENT'],testdf['DC_SPEND_MON_03_RST'],testdf['DC_SPEND_MON_03_HTL']
del testdf['DC_SPEND_MON_03_HBY'],testdf['DC_SPEND_MON_03_CARE'],testdf['DC_SPEND_MON_03_TRL'],testdf['DC_SPEND_MON_03_JER']
del testdf['DCGODSPN02'],testdf['DC_SPEND_MON_02_MED'],testdf['DC_SPEND_MON_02_HMD'],testdf['DC_SPEND_MON_02_ATM']
del testdf['DCBADSPN02'],testdf['DC_SPEND_MON_02_ENT'],testdf['DC_SPEND_MON_02_RST'],testdf['DC_SPEND_MON_02_HTL']
del testdf['DC_SPEND_MON_02_HBY'],testdf['DC_SPEND_MON_02_CARE'],testdf['DC_SPEND_MON_02_TRL'],testdf['DC_SPEND_MON_02_JER']
del testdf['DCGODSPN01'],testdf['DC_SPEND_MON_01_MED'],testdf['DC_SPEND_MON_01_HMD'],testdf['DC_SPEND_MON_01_ATM']
del testdf['DCBADSPN01'],testdf['DC_SPEND_MON_01_ENT'],testdf['DC_SPEND_MON_01_RST'],testdf['DC_SPEND_MON_01_HTL']
del testdf['DC_SPEND_MON_01_HBY'],testdf['DC_SPEND_MON_01_CARE'],testdf['DC_SPEND_MON_01_TRL'],testdf['DC_SPEND_MON_01_JER']

In [69]:
testdf.shape

(200000, 221)

In [70]:
# diff of salary - EMI
testdf['HLEMIPERC03'] = testdf.fillna(0)['SAL_MON_03']/testdf.fillna(0)['HL_EMI']
testdf['OLEMIPERC03'] = testdf.fillna(0)['SAL_MON_03']/testdf.fillna(0)['SCRUB_EMI']

testdf['HLEMIPERC02'] = testdf.fillna(0)['SAL_MON_02']/testdf.fillna(0)['HL_EMI']
testdf['OLEMIPERC02'] = testdf.fillna(0)['SAL_MON_02']/testdf.fillna(0)['SCRUB_EMI']

testdf['HLEMIPERC01'] = testdf.fillna(0)['SAL_MON_01']/testdf.fillna(0)['HL_EMI']
testdf['OLEMIPERC01'] = testdf.fillna(0)['SAL_MON_01']/testdf.fillna(0)['SCRUB_EMI']

#replace Nan with 0
testdf['HLEMIPERC03'] = testdf.HLEMIPERC03.fillna(0)
testdf['OLEMIPERC03'] = testdf.OLEMIPERC03.fillna(0)
testdf['HLEMIPERC02'] = testdf.HLEMIPERC02.fillna(0)
testdf['OLEMIPERC02'] = testdf.OLEMIPERC02.fillna(0)
testdf['HLEMIPERC01'] = testdf.HLEMIPERC01.fillna(0)
testdf['OLEMIPERC01'] = testdf.OLEMIPERC01.fillna(0)

In [71]:
# Columns to delete
del testdf['ZIP_CODE_FINAL'],testdf['TOP_100_CORP'],testdf['TOP_800_CORP']

In [72]:
testdf.shape

(200000, 224)

In [73]:
# There are now 228 variables in the training set data
nonnumcols = testdf.dtypes[testdf.dtypes == "object"].index
print (len(nonnumcols))
#print (nonnumcols)

33


In [74]:
print ('Before One hit encoding', testdf.shape)

Before One hit encoding (200000, 224)


In [75]:
testdf = pd.get_dummies(testdf, 
                         columns=['ACT_TYPE','GENDER','LEGAL_ENTITY','CC_HOLD','CC_ACTIVE','DC_HOLD','DC_ACTIVE',
                                  'FINANCE_MON_01','JOBS_MON_01','LIFESTYLE_MON_01','INMON_01KET_MON_01',
                                  'TRAVEL_MON_01','CHANNEL_CLICK_DISP','DESIGNATION_FINAL','TOP14_CITY','TOP9_CITY',
                                  'PA_PQ_TAG','NEFT_CC_CATEGORY','NEFT_DC_CATEGORY','TPT_DC_CATEGORY_MON_01',
                                  'TPT_CC_CATEGORY_MON_01','IMPS_CC_CATEGORY_MON_01','CARD_AUTOMOBILE_MON_01',
                                  'CARD_HOTEL_MON_01','CARD_JEWELLERY_MON_01','CARD_RESTAURANT_MON_01',
                                  'CARD_TRAVEL_MON_01','CARD_PRSNL_CARE_MON_01','CARD_HOBBY_MON_01',
                                  'CARD_HOME_DECOR_MON_01','CARD_MEDICAL_MON_01','CARD_ENTMNT_MON_01',
                                  'TOP_CORP_TAG'], drop_first=True)
print ('After One hot encoding',testdf.shape)

After One hot encoding (200000, 454)


In [77]:
print (traindf.shape, testdf.shape)

(300000, 470) (200000, 454)


In [78]:
listA = list(traindf.columns.values)
listB = list(testdf.columns.values)

In [79]:
print (len(listA), len(listB))

470 454


In [80]:
"""
for item in listB:
    if item not in listA:
        print (item)
        """

'\nfor item in listB:\n    if item not in listA:\n        print (item)\n        '

In [81]:
testdf['LEGAL_ENTITY_FIRM'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_103K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_107K'] = 0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_112K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_115K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_121K'] = 0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_122K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_1250K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_129K'] = 0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_134K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_182K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_183K'] = 0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_192K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_2000K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_205K'] = 0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_233K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_241K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_246K']=0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_247K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_258K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_292K']=0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_333K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_400K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_476K']=0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_583K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_667K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_700K']=0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_833K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_94K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_97K']=0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_98K'],testdf['NEFT_CC_CATEGORY_WEDDING'],testdf['NEFT_DC_CATEGORY_RETURN']=0,0,0
testdf['TPT_DC_CATEGORY_MON_01_OPERATION'],testdf['IMPS_CC_CATEGORY_MON_01_JEWELLERY']=0,0

In [82]:
"""
for item in listB:
    if item not in listA:
        print (item)
"""

'\nfor item in listB:\n    if item not in listA:\n        print (item)\n'

In [83]:
traindf['LEGAL_ENTITY_PROPRIETERSHIP'],traindf['LEGAL_ENTITY_TRUSTS'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_118K']=0,0,0
traindf['PA_PQ_TAG_PQ_WITH_INCOME_124K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_130K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_136K']=0,0,0
traindf['PA_PQ_TAG_PQ_WITH_INCOME_143K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_144K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_158K']=0,0,0
traindf['PA_PQ_TAG_PQ_WITH_INCOME_185K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_188K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_196K']=0,0,0
traindf['PA_PQ_TAG_PQ_WITH_INCOME_260K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_317K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_3333K']=0,0,0
traindf['PA_PQ_TAG_PQ_WITH_INCOME_407K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_4167K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_600K']=0,0,0
traindf['NEFT_CC_CATEGORY_CC_PAYMENT'],traindf['NEFT_DC_CATEGORY_BUILDER']=0,0

# Restart after train and testdf extract

In [2]:
#traindf = pd.read_csv('/data/traindf.csv')
traindf = pd.read_csv('/Users/Vishy/Files/AVDatahack/CrossSell/ProData/traindf.csv')

In [3]:
#testdf = pd.read_csv('/data/testdf.csv')
testdf = pd.read_csv('/Users/Vishy/Files/AVDatahack/CrossSell/ProData/testdf.csv')

In [4]:
print (traindf.shape, testdf.shape)

(300000, 490) (200000, 489)


In [9]:
del traindf['STMT_CON_DAE_CLOSED_MON_01'],traindf['STMT_CON_DAE_ACTIVE_MON_01'],traindf['MER_EMI_CLOSED_MON_01']
del traindf['PM_FD_MON_04'],traindf['PM_FD_MON_02'],traindf['LFP_MON_03'],traindf['CDL_MON_03'],traindf['LFP_MON_01']
del traindf['CDL_MON_01'],traindf['LAA_MON_03'],traindf['CDL_MON_02'],traindf['LFP_MON_02'],traindf['LAA_MON_02']
del traindf['GL_MON_03'],traindf['LAA_MON_01'],traindf['GL_MON_01'],traindf['GL_MON_02'],traindf['TWL_MON_02']
del traindf['TWL_MON_01'],traindf['TWL_MON_03'],traindf['CAR_MON_02'],traindf['PM_FD_MON_01'],traindf['CAR_MON_01']
del traindf['CAR_MON_03'],traindf['PL_MON_02'],traindf['PL_MON_03'],traindf['PL_MON_01'],traindf['HL_EMI']
del traindf['TPT_DC_AMOUNT_MON_01'],traindf['TPT_DC_TXN_MON_01']
traindf.shape

(300000, 460)

In [5]:
allcols = list(traindf.columns.values)
print (len(allcols))

490


In [6]:
testdf['RESPONDERS']=0

In [7]:
traindf = traindf[allcols]
testdf = testdf[allcols]
del testdf['RESPONDERS']

In [8]:
target_train = traindf['RESPONDERS'].values
id_test = testdf['CUSTOMER_ID'].values

train = np.array(traindf.drop(['RESPONDERS','CUSTOMER_ID'], axis = 1))
test = np.array(testdf.drop(['CUSTOMER_ID'], axis = 1))

xgb_preds = []

In [9]:
traindf.RESPONDERS.value_counts()

0    295388
1      4612
Name: RESPONDERS, dtype: int64

In [10]:
"""
from sklearn.metrics import accuracy_score
def liftgain(df: pd.DataFrame, actual_col: str, predicted_col:str, probability_col:str):
    df.sort_values(by=probability_col, ascending=False, inplace=True)
    subset = df[df[predicted_col] == True]
    rows = []
    for group in np.array_split(subset, 10):
        score = accuracy_score(group[actual_col].tolist(),group[predicted_col].tolist(),normalize=False)
        rows.append({'NumCases': len(group), 'NumCorrectPredictions': score})
        lift = pd.DataFrame(rows)
        lift['liftgain'] = lift['NumCorrectPredictions']/lift['NumCases'].astype(float)
        del lift['NumCorrectPredictions']
        del lift['NumCases']
    return lift
"""
def liftscr(y_target, y_predicted):
    scr = lift_score(y_target, y_predicted)
    return scr

def lift_xgb(preds, d_train):
    #labels = dtrain.get_label()
    labels = d_train
    #df = pd.DataFrame({'acu': labels, 'proba': preds})
    #df['pred'] = df.proba.apply(lambda x: np.where(x >= 0.51,1,0))
    #lift_score = liftgain(df,'acu','pred','proba')
    lifts = liftscr(labels, preds)
    lift_score = lifts/10
    return [('lift', lift_score)]

In [10]:
K = 3
kf = KFold(n_splits = K, random_state = 42, shuffle = True)

In [11]:
from sklearn.metrics import recall_score
start = time.time()

for train_index, test_index in kf.split(train):
    train_X, valid_X = train[train_index], train[test_index]
    train_y, valid_y = target_train[train_index], target_train[test_index]
    xgb_params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.5, 'colsample_bytree': 0.9, 
                  'objective': 'binary:logistic', 'eval_metric': 'ndcg@10000', 'seed': 99, 'silent': True}

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(test)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train, 5000,  watchlist, maximize=True, #feval=lift_xgb,
                      verbose_eval=100, early_stopping_rounds=50)
                        
    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))

end = time.time()
print ('Total Time taken for ',K,' cross validation is', end-start)

[0]	train-ndcg@10000:0.30776	valid-ndcg@10000:0.402563
Multiple eval metrics have been passed: 'valid-ndcg@10000' will be used for early stopping.

Will train until valid-ndcg@10000 hasn't improved in 50 rounds.
[100]	train-ndcg@10000:0.445117	valid-ndcg@10000:0.520057
[200]	train-ndcg@10000:0.473933	valid-ndcg@10000:0.546031
[300]	train-ndcg@10000:0.498946	valid-ndcg@10000:0.565833
[400]	train-ndcg@10000:0.528186	valid-ndcg@10000:0.578954
Stopping. Best iteration:
[442]	train-ndcg@10000:0.539771	valid-ndcg@10000:0.58404

[0]	train-ndcg@10000:0.266619	valid-ndcg@10000:0.372185
Multiple eval metrics have been passed: 'valid-ndcg@10000' will be used for early stopping.

Will train until valid-ndcg@10000 hasn't improved in 50 rounds.
[100]	train-ndcg@10000:0.441999	valid-ndcg@10000:0.520558
[200]	train-ndcg@10000:0.476251	valid-ndcg@10000:0.548296
[300]	train-ndcg@10000:0.502896	valid-ndcg@10000:0.553878
[400]	train-ndcg@10000:0.529084	valid-ndcg@10000:0.565607
[500]	train-ndcg@10000:0.55

In [12]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+=xgb_preds[j][i]
    preds.append(sum / K)

out = pd.DataFrame({'CUSTOMER_ID': id_test, 'RESPONDERS': preds})

In [13]:
out.to_csv("pred5.csv", index=False)

In [95]:
#traindf.to_csv("traindf.csv", index=False)

In [96]:
#testdf.to_csv("testdf.csv", index=False)

In [97]:
#feature_importance = model.get_fscore()
#feats = pd.DataFrame.from_dict(feature_importance, orient='index')
#feats.to_csv('feats.csv')